In [ ]:
# numerical analyses libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import natsort
import scipy
from scipy.signal import sosfiltfilt, butter, hilbert
import scipy.signal as signal
# libraries for interacting/setting directories for the data
import os
import mne
from bids.layout import BIDSLayout
from pathlib import Path
%matplotlib inline


# consulting the PyBIDS documentation...

general functions:
- using layout.get_tasks() to identify strings ; layout.to_df() ; layout.get_tasks() ; using dir(anyclass...) to get all applicable functions, besides the 'dunder' functions
- help(layout) ; print(bids.__version__)  to get version

# creating directory structure for accessing and processing all .edf files; Github solution to empty solutions issue previously (https://github.com/bids-standard/pybids/issues/978)

In [ ]:
data_path = 'C:\\Users\\josep\\Music&Topology' 

In [ ]:
# initialising the layout
layout = BIDSLayout(data_path,validate=True)
layout

layout.get(return_type="id", target="subject")

In [ ]:
#creating subject-level directory paths w/ Github solution
subject_dirs = [Path(data_path) / 'all-subjects' / f'sub-{subject}' for subject in layout.get_subjects()]
subject_dirs

In [ ]:
# 招 Pathlib documentation,
Path('C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-01\\eeg\\sub-01_task-classicalMusic_eeg.edf').exists()

#since my '.edf' files do indeed exist (lol), going forward with making dictionaries, epoching, all that good stuff

In [ ]:
# Starting from the root of dataset, search for all .edf files w/Pathlib, since pybids didn't work
all_edf_files = list(Path(layout.root).rglob('*.edf'))

# Print all findable .edf files, using Pathlib from above
for file in all_edf_files:
    print(file)

In [ ]:
# Extract unique subject directories, or the parents of the EEG directories from all_edf_files
all_subject_dirs_with_edf = list({file.parent.parent for file in all_edf_files})

for dir in all_subject_dirs_with_edf:
    print(dir.name)

# pre-processing: filtering, selecting subjects, ICA

- general outline: 

  1. select subjects from visual inspection of channels filtered for delta, theta and alpha bands, looking for subjects with the least inter-subject variability in EEG patterns.
  2. from selected subjects, ICA loop from 1-30 hz for raw-->raw-with-ICA-weights data; continuous data & still not trial-segmented
  3. re-filter the raw-with-ICA-weights data for each subject for delta, theta and alpha bands, 0.5-13 hz
  4. filter as well for four bands, for ' α (1–7 Hz), β (8–13 Hz), θ (14–30 Hz), and γ bands (30–45 Hz)' for the DL model from 'EEG-Based Emotion Classification...'
  3. extract & visualize the non-EEG channels' data
  4. follow musical moment epoching procedure from pg. 7-8 from notebook

  #keep for later, after the filtering and after the trial segmentations
  #your actual epochs, for musical moments
  
  epochs_output_folder = "C:\\Users\\josep\\Music&Topology\\Analysis\\epochs_data"
  if not os.path.exists(epochs_output_folder):
    os.makedirs(epochs_output_folder)

In [ ]:
event_id = {
    'ExpStart': 1,
    'TTL': 10,
    'TTL-relay': 47,
    'Artifact:pulse': 265,
    'Trial start': 768,
    'ecg:Q-wave-onset, QRS-onset': 1283,
    'ecg:S-wave-onset, S-wave-peak': 1285,
    'unused': 34053,
}

#from participants.tsv

#participant_id	age	sex
#sub-01	29	F
#sub-02	21	F
#sub-03	26	M
#sub-04	25	M
#sub-05	25	M
#sub-06	26	M
#sub-07	27	F
#sub-08	26	F
#sub-09	26	F
#sub-10	27	M
#sub-11	20	F
#sub-12	23	F
#sub-13	20	M
#sub-14	22	F
#sub-15	23	M
#sub-16	23	M
#sub-17	22	F
#sub-18	21	F
#sub-19	24	M
#sub-20	27	M
#sub-21	28	M

# continuation from 10/24: visual inspection

In [ ]:
# minimize this; define 'preprocess_data' function

# Define frequency bands
delta_band = (1, 4)
theta_band = (4, 7)
alpha_band = (8, 13)

# Define a combined filter band that spans delta, theta, and alpha frequencies
combined_band = (delta_band[0], alpha_band[1])

def preprocess_data(subject_id):
    # Load the EEG data
    file_name = f"C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-{subject_id:02d}\\eeg\\sub-{subject_id:02d}_task-classicalMusic_eeg.edf"
    raw = mne.io.read_raw_edf(file_name, preload=True)
    
    # Bandpass filter the data for the combined frequency band (theta + alpha)
    raw_combined = raw.copy().filter(l_freq=combined_band[0], h_freq=combined_band[1])
    
    return raw_combined

In [ ]:
# minimize this; define 'preprocess_data_and_plot' function
# automating to have each plot for each selected channel saved as a PNG, saved in a folder for that respective subject, then upload to GPT-4 for a 'second take' noise analysis
import os


def preprocess_data_and_plot(subject_id):
    # Define the channels of interest
    channels_of_interest = ['Fp1', 'Fp2', 'F3', 'F4', 'F7', 'F8', 'P3', 'P4', 'P7', 'P8', 'T7', 'T8', 'Fz', 'Cz', 'Pz', 'Oz', 'FC1', 'FC2', 'CP1', 'CP2', 'FC5', 'FC6', 'CP5', 'CP6', 'TP9', 'TP10', 'POz']

    # Define the path to the preprocessed data
    # Load the preprocessed data
    raw_combined = preprocess_data(subject_id)

    # Define time range
    start, stop = raw_combined.time_as_index([30, 2000])

    # Directory where the plots will be saved
    save_dir = f"C:\\Users\\josep\\Music&Topology\\Analysis\\Analysis\\r[{subject_id}];)"
    
    # Create the directory if it doesn't exist
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # Loop through each channel and plot
    for channel in channels_of_interest:
        # Extracting data for the channel
        combined_data, _ = raw_combined[channel, start:stop]

        # Plotting
        plt.figure(figsize=(15, 6))
        plt.plot(raw_combined.times[start:stop], combined_data.T)
        plt.title(f'Theta and Alpha Filtered EEG Data for Channel {channel} (30s to 2000s)')
        plt.xlabel('Time (s)')
        plt.ylabel('EEG amplitude (uV)')
        plt.tight_layout()
        
        # Save the plot to the specified directory with the channel name
        plt.savefig(os.path.join(save_dir, f"{channel}_filtered.png"))
        
        plt.close()  # Close the current plot before moving to the next

# usage for a single subject; change () number to subject_id
preprocess_data_and_plot(21)


# continuation since 10/27: steps 1-4

# step 1: selected subjects

In [ ]:
# List of selected subjects from the above visual inspection step; 'sub_using' folder has the PNG's for these
selected_subjects = ['02', '04', '05', '10', '17', '19', '21']

# Filter out directories that match the 'sub-XX' format and are in the list of selected subjects
selected_subject_dirs = [dir for dir in all_subject_dirs_with_edf if 'sub-' in dir.name and any(sub in dir.name for sub in selected_subjects)]

# Print the directories of the selected subjects
print(selected_subject_dirs)  

for dir in selected_subject_dirs:
    print(dir.name)

In [ ]:
# exploring the non-EEG channels w/in the '...channels.tsv' file of each subject, before conducting ICA and following analyses

# Load the EEG data for one of the subjects (you can change this to any other subject's file)
edf_file = Path("C:/Users/josep/Music&Topology/all-subjects/sub-16/eeg/sub-16_task-classicalMusic_eeg.edf")
raw = mne.io.read_raw_edf(edf_file, preload=True)

# List of additional channels
additional_channels = ['ECG', 'ft_valence', 'ft_arousal', 'ft_x', 'ft_y', 'ft_ghostvalence', 
                       'ft_ghostarousal', 'music', 'trialtype', 'sams_valence', 'sams_arousal', 
                       'sams_valencert', 'sams_arousalrt', 'nback_stimuli', 'nback_keypress']

# Pick only the additional channels for visualization
raw.pick_channels(additional_channels)

# Plot the time courses of these channels
raw.plot(start=0, duration=60)  # Displaying 60 seconds for illustration, adjust as needed

In [ ]:
# Segmentation/Epoching the time series, based on trial type and piece of music
# The value of the music channel indicates which piece of music (from the stimuli folder) was played to the participant in a given trial. To convert from the values stored in this channel to the music channel: 1) multiply the value by 20, 2) convert to a string, 3) the file name is then constructed from the resulting 3-element number. For example, if the number is 282 this indicates file 2-8_2.wav from the stimuli folder.

# Extract data for the trialtype and music channels
trialtype_data = raw.copy().pick_channels(['trialtype']).get_data()[0]
music_data = raw.copy().pick_channels(['music']).get_data()[0]

# Let's plot these channels to visualize the changes
times = raw.times
plt.figure(figsize=(15, 6))

plt.subplot(2, 1, 1)
plt.plot(times, trialtype_data)
plt.title('trialtype')
plt.xlabel('Time (s)')
plt.ylabel('Value')

plt.subplot(2, 1, 2)
plt.plot(times, music_data)
plt.title('music')
plt.xlabel('Time (s)')
plt.ylabel('Value')

plt.tight_layout()
plt.show()


# step 2: ICA, filtering

In [ ]:
# loop for filtering & ICA on the raw data-->raw-with-ICA-weights

channels_of_interest = ['F3', 'F4', 'F7', 'F8', 'P3', 'P4', 'P7', 'P8', 
                        'T7', 'T8', 'Fz', 'Cz', 'Pz', 'Oz', 'FC1', 'FC2', 'CP1', 'CP2', 
                        'FC5', 'FC6', 'CP5', 'CP6', 'TP9', 'TP10', 'POz']

# Create output folder if it doesn't exist
output_folder = "C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\raw-with-ICA-weights" 
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Create ICA properties plots output folder
ica_plots_folder = "C:\\Users\\josep\\Music&Topology\\all work ;)\\images\\ICAproperties"
if not os.path.exists(ica_plots_folder):
    os.makedirs(ica_plots_folder)

# Iterate over the selected subject directories
for dir in selected_subject_dirs:
    edf_files_in_dir = [f for f in dir.glob('eeg/*.edf') if 'classicalMusic' in f.name]
    for edf_file in edf_files_in_dir:
        # Load the EEG data
        raw = mne.io.read_raw_edf(edf_file, preload=True)
     
        # Notch filter to remove 50 Hz line noise
        raw.notch_filter([50], filter_length='auto', phase='zero')
        
        # Band-pass filter for delta, theta, and alpha bands
        raw.filter(l_freq=1, h_freq=30)
         
        # Reference the EEG data to its average
        raw.set_eeg_reference(ref_channels='average')
    
        # Explicitly drop non-EEG channels
        non_eeg_channels = ['ECG', 'ft_valance', 'ft_arousal', 'ft_x', 'ft_y', 'ft_ghostvalence', 
                            'ft_ghostarousal', 'music', 'trialtype', 'sams_valence', 'sams_arousal', 
                            'sams_valencert', 'sams_arousalrt', 'nback_stimuli', 'nback_keypress']

        raw.drop_channels(non_eeg_channels)

        # Set the standard 10/20 montage
        montage = mne.channels.make_standard_montage('standard_1020')
        raw.set_montage(montage)

        raw.pick_channels(channels_of_interest, ordered=False)
        
        # Use ICA to remove artifacts
        ica = mne.preprocessing.ICA(n_components=25, random_state=97, max_iter=800)
        ica.fit(raw)
        
        # Plot and save the ICA components' properties including the power spectrum for GPT-4 'second eye' inspection
        for idx in range(25):
            fig = ica.plot_properties(raw, picks=idx, show=False)
            subject_id = dir.name.split('-')[-1]
            fig_name = f"ICA_sub{subject_id}-plot{idx+1}.png"
            fig[0].savefig(os.path.join(ica_plots_folder, fig_name))
            plt.close(fig[0])  # close the figure to free up memory
        
        # Exclude selected components
        ica.exclude = [int(i) for i in input(f"Enter indices of components to exclude for {edf_file.name} (comma-separated): ").split(",")]

        # Apply the ICA to the raw data
        raw = ica.apply(raw)

        # Save the preprocessed data with the specified filename
        raw.save(os.path.join(output_folder, f"{edf_file.stem}_raw-with-ICA-weights.fif"), overwrite=True)

In [ ]:
# process raw-with-ICA-weights data for DL model


In [ ]:
# visualization check

In [ ]:
# process raw-with-ICA-weights data for analyses

import mne
import os

# List of subjects' raw data with applied ICA weights
subjects_files = [
    "C:/Users/josep/Music&Topology/all work ;)/Analysis1/raw-with-ICA-weights/raw-with-ICA-weights/sub-02_task-classicalMusic_eeg_raw-with-ICA-weights.fif",
    "C:/Users/josep/Music&Topology/all work ;)/Analysis1/raw-with-ICA-weights/raw-with-ICA-weights/sub-04_task-classicalMusic_eeg_raw-with-ICA-weights.fif",
    "C:/Users/josep/Music&Topology/all work ;)/Analysis1/raw-with-ICA-weights/raw-with-ICA-weights/sub-05_task-classicalMusic_eeg_raw-with-ICA-weights.fif",
    "C:/Users/josep/Music&Topology/all work ;)/Analysis1/raw-with-ICA-weights/raw-with-ICA-weights/sub-10_task-classicalMusic_eeg_raw-with-ICA-weights.fif",
    "C:/Users/josep/Music&Topology/all work ;)/Analysis1/raw-with-ICA-weights/raw-with-ICA-weights/sub-17_task-classicalMusic_eeg_raw-with-ICA-weights.fif",
    "C:/Users/josep/Music&Topology/all work ;)/Analysis1/raw-with-ICA-weights/raw-with-ICA-weights/sub-19_task-classicalMusic_eeg_raw-with-ICA-weights.fif",
    "C:/Users/josep/Music&Topology/all work ;)/Analysis1/raw-with-ICA-weights/raw-with-ICA-weights/sub-21_task-classicalMusic_eeg_raw-with-ICA-weights.fif"
]

# Create output folder if it doesn't exist                                                                                                                                          
output_folder = "C:/Users/josep/Music&Topology/all work ;)/Analysis1/processed-for-analysis"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

for subject_file in subjects_files:
    # Load the raw data
    raw = mne.io.read_raw_fif(subject_file, preload=True)
    
    # Notch filter to remove 50 Hz line noise
    raw.notch_filter([50], filter_length='auto', phase='zero')
    
    # Band-pass filter for delta, theta, and alpha bands
    raw.filter(l_freq=0.5, h_freq=13)
     
    # Reference the EEG data to its average
    raw.set_eeg_reference(ref_channels='average')
    
   # Extract the filename without the path and extension
    filename = os.path.basename(subject_file).split('.')[0]

    # Construct the new filename with "filtered-and-referenced.fif" added
    new_filename = f"{filename}-filtered-and-referenced_analysis.fif"

    # Create the full output path in the output_folder
    output_path = os.path.join(output_folder, new_filename)

    # Save the modified data to the new location
    raw.save(output_path)

print("All files have been filtered, re-referenced, and saved in the processed-for-analysis folder.")


In [ ]:
# visualization check

'''
import os
import mne
import matplotlib.pyplot as plt

def preprocess_data_and_plot(subject_id):
    # Define the path to the processed data for the subject
    file_path = f"C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\processed-for-analysis\\sub-{subject_id:02d}_task-classicalMusic_eeg_raw-with-ICA-weights-filtered-and-referenced_analysis.fif"
    
    # Load the processed data
    raw_processed = mne.io.read_raw_fif(file_path, preload=True)
    
    # Define the channels of interest
    channels_of_interest = ['F3', 'F4', 'F7', 'F8', 'P3', 'P4', 'P7', 'P8', 'T7', 'T8', 'Fz', 'Cz', 'Pz', 'Oz', 'FC1', 'FC2', 'CP1', 'CP2', 'FC5', 'FC6', 'CP5', 'CP6', 'TP9', 'TP10', 'POz']

    # Define time range
    start, stop = raw_processed.time_as_index([10, 2000])

    # Directory where the plots will be saved
    save_dir = "C:\\Users\\josep\\Music&Topology\\all work ;)\\images\\processing check"
    
    # Create the directory if it doesn't exist
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # Loop through each channel and plot
    for channel in channels_of_interest:
        # Extracting data for the channel
        combined_data, _ = raw_processed[channel, start:stop]

        # Plotting
        plt.figure(figsize=(15, 6))
        plt.plot(raw_processed.times[start:stop], combined_data.T)
        plt.title(f'Theta and Alpha Filtered EEG Data for Channel {channel} (30s to 2000s)')
        plt.xlabel('Time (s)')
        plt.ylabel('EEG amplitude (uV)')
        plt.tight_layout()
        
        # Save the plot to the specified directory with the channel name
        plt.savefig(os.path.join(save_dir, f"sub-{subject_id:02d}_{channel}_filtered.png"))
                
# usage for a single subject; change () number to subject_id
preprocess_data_and_plot(2)
'''

# step 3: non-EEG channels' visualization

In [ ]:
# loop for visualization of non-EEG channels; clarification for next steps

import os

# Iterate over the selected subject directories
for dir in selected_subject_dirs:
    # Get only the classical music EDF files
    classical_music_files = list(dir.glob('eeg/*task-classicalMusic_eeg.edf'))
    
    for edf_file in classical_music_files:
        # Load the data
        raw = mne.io.read_raw_edf(edf_file, preload=True)
        
        # Get the indices of the non-EEG channels
        non_eeg_channels = ['ft_valance', 'ft_arousal', 'music', 'trialtype']
        channel_indices = [raw.ch_names.index(ch) for ch in non_eeg_channels if ch in raw.ch_names]

        # For each non-EEG channel, plot the data using plt
        for ch_idx in channel_indices:
            data, times = raw[ch_idx, :]
    
            plt.figure(figsize=(10, 4))
            plt.plot(times, data.T)
            plt.title(f"{raw.ch_names[ch_idx]} for {edf_file.stem}")
            plt.xlabel('Time (s)')
            plt.ylabel('Value')
            plt.tight_layout()
            plt.show()
            
'''
music	stimuli	V	good	
The value of the music channel indicates which piece of music (from the stimuli folder) was played to the participant
in a given trial. To convert from the values stored in this channel to the music channel: 1) multiply the value by 20, 2) 
convert to a string, 3) the file name is then constructed from the resulting 3-element number. For example, 
if the number is 282 this indicates file 2-8_2.wav from the stimuli folder.
'''

# step 5: visualization of the trialsegs, setting up for epochs; first, for music-and-reporting


In [ ]:
The raw data and their corresponding timestamps/event markers for the music-and-reporting trialsegs are as follows:

 raw file:
 'C:\Users\josep\Music&Topology\all-subjects\sub-02\eeg\sub-02_task-classicalMusic_eeg.edf'
corresponding timestamps for epochs:
17.092-175.484 (s) for 'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data\\sub02\\sub02_music-and-reporting_p3-epo.fif'
175.484-308.868 (s) for 'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data\sub02\\sub02_music-and-reporting_p6-or-p2-epo.fif'

raw file:
'C:\Users\josep\Music&Topology\all-subjects\sub-04\eeg\sub-04_task-classicalMusic_eeg.edf'
corresponding timestamps for epochs:
175.555-337.511 (s) for 'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data\\sub04\\sub04_music-and-reporting_p5-undecided-epo.fif'
337.511-472.401 (s) for 'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data\\sub04\\sub04_music-and-reporting_p6-or-p2-epo.fif'
605.756-761.398 (s) for 'C:\\Users\\josep\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data\\sub04\\sub04_music-and-reporting_p3-epo.fif'
1059.81-1225.402 (s) for 'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data\\sub04\\sub04_music-and-reporting_p5-undecided-epo.fif'

raw file:
'C:\Users\josep\Music&Topology\all-subjects\sub-05\eeg\sub-05_task-classicalMusic_eeg.edf'
corresponding timestamps for epochs:
14.594-148.165 (s) for 'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data\\sub05\\sub05_music-and-reporting_p6-or-p2-epo.fif'
282.772-448.714 (s) for 'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data\\sub05\\sub05_music-and-reporting_p5-undecided-epo.fif'
908.811-1065.856 (s) for 'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data\\sub05\\sub05_music-and-reporting_p3-epo.fif'
1224.351-1389.993 (s) for 'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data\\sub05\\sub05_music-and-reporting_p5-undecided-epo.fif'

raw file:
'C:\Users\josep\Music&Topology\all-subjects\sub-10\eeg\sub-10_task-classicalMusic_eeg.edf'
corresponding timestamps for epochs:
14.906-149.429 (s) for 'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data\\sub10\\sub10_music-and-reporting_p6-or-p2-epo.fif'
447.09-602.681 (s) for 'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data\\sub10\\sub10_music-and-reporting_p3-epo.fif'
769.324-932.846 (s) for C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data\\sub10\\sub10_music-and-reporting_p5-undecided-epo.fif'
1089.656-1254.697 (s) for C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data\\sub10\\sub10_music-and-reporting_p5-undecided-epo.fif'

raw file:
'C:\Users\josep\Music&Topology\all-subjects\sub-17\eeg\sub-17_task-classicalMusic_eeg.edf'
corresponding timestamps for epochs:
14.109-172.254 (s) for 'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data\\sub17\\sub17_music-and-reporting_p3-epo.fif'
172.254-339.316 (s) for 'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data\\sub17\\sub17_music-and-reporting_p5-undecided-epo.fif'
628.262-791.817 (s) for 'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data\\sub17\\sub17_music-and-reporting_p5-undecided-epo.fif'
791.817-925.556 (s) for 'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data\\sub17\\sub17_music-and-reporting_p6-or-p2-epo.fif'

raw file:
'C:\Users\josep\Music&Topology\all-subjects\sub-19\eeg\sub-19_task-classicalMusic_eeg.edf'
corresponding timestamps for epochs:
179.071-344.363 (s) for 'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data\\sub19\\sub19_music-and-reporting_p5-undecided-epo.fif'
502.826-637.032 (s) for 'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data\\sub19\\sub19_music-and-reporting_p6-or-p2-epo.fif'
637.032-794.395 (s) for  'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data\\sub19\\sub19_music-and-reporting_p3-epo.fif'
1124.542-1288.438 (s) for 'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data\\sub19\\sub19_music-and-reporting_p5-undecided-epo.fif'

raw file:
'C:\Users\josep\Music&Topology\all-subjects\sub-21\eeg\sub-21_task-classicalMusic_eeg.edf'
corresponding timestamps for epochs:
180.259-345.484 (s) for 'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data\\sub21\\sub21_music-and-reporting_p5-undecided-epo.fif'
502.01-636.067 (s) for 'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data\\sub21\\sub21_music-and-reporting_p6-or-p2-epo.fif''
636.067-791.376 (s) for 'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data\\sub21\\sub21_music-and-reporting_p3-epo.fif'
1121.608-1285.497 (s) for 'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data\\sub21\\sub21_music-and-reporting_p5-undecided-epo.fif'

In [ ]:
import os
import mne
import matplotlib.pyplot as plt

# List of music-and-reporting trialsegs files to visualize
epoch_files = ['C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data\\sub02\\sub02_music-and-reporting_p3-epo.fif',
               'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data\sub02\\sub02_music-and-reporting_p6-or-p2-epo.fif',
               'C:\\Users\\josep\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data\\sub04\\sub04_music-and-reporting_p3-epo.fif',
               'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data\\sub04\\sub04_music-and-reporting_p5-undecided-epo.fif',
               'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data\\sub04\\sub04_music-and-reporting_p6-or-p2-epo.fif',
               'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data\\sub05\\sub05_music-and-reporting_p3-epo.fif',
               'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data\\sub05\\sub05_music-and-reporting_p5-undecided-epo.fif',
               'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data\\sub05\\sub05_music-and-reporting_p6-or-p2-epo.fif',
               'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data\\sub10\\sub10_music-and-reporting_p3-epo.fif',
               'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data\\sub10\\sub10_music-and-reporting_p5-undecided-epo.fif',
               'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data\\sub10\\sub10_music-and-reporting_p6-or-p2-epo.fif',
               'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data\\sub17\\sub17_music-and-reporting_p3-epo.fif',
               'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data\\sub17\\sub17_music-and-reporting_p5-undecided-epo.fif',
               'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data\\sub17\\sub17_music-and-reporting_p6-or-p2-epo.fif',
               'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data\\sub19\\sub19_music-and-reporting_p3-epo.fif',
               'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data\\sub19\\sub19_music-and-reporting_p5-undecided-epo.fif',
               'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data\\sub19\\sub19_music-and-reporting_p6-or-p2-epo.fif',
               'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data\\sub21\\sub21_music-and-reporting_p3-epo.fif',
               'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data\\sub21\\sub21_music-and-reporting_p5-undecided-epo.fif',
               'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data\\sub21\\sub21_music-and-reporting_p6-or-p2-epo.fif'
]

for epoch_file in epoch_files:
    # Load the epoch
    epochs = mne.read_epochs(epoch_file, preload=True)
    
    # Check if the channels of interest are present in the data
    channels_of_interest = ['ft_valance', 'ft_arousal', 'music']
    available_channels = [ch for ch in channels_of_interest if ch in epochs.ch_names]
    
    # Plot the data for each channel of interest
    for ch in available_channels:
        # Retrieve the data and times
        data = epochs.get_data(picks=ch)
        times = epochs.times
        
        # Plot
        plt.figure(figsize=(15, 4))
        plt.plot(times, data[0, 0, :])  # Plotting the first epoch's data for the channel
        plt.title(f"{ch} for {os.path.basename(epoch_file)}")
        plt.xlabel('Time (s)')
        plt.ylabel('Value')
        plt.xticks(np.arange(min(times)*1000, (max(times)+0.1)*1000, 100))  # For every 100 milliseconds
        plt.grid(True)
        plt.tight_layout()
        plt.show()


In [ ]:
'''
layout to troubleshoot discrepancy in valance/arousal plots on our end vs. the article's Results Figure 2
1) Check the range of 'ft_valance' and 'ft_arousal' values across all subjects and trials to understand the current scaling.
2) If the values do not range from 0 to 1, calculate the scaling factor needed to adjust them to this range.
3) Apply the scaling factor to the 'ft_valance' and 'ft_arousal' values in your plots to ensure they match the expected scale.

read the EDF file header
'''

In [ ]:
# 1)
# script that iterates over each subject, loads their EDF file, and extracts the 'ft_valance' and 'ft_arousal' data to find the minimum and maximum values across all subjects and trials. 
# give us a better understanding of the scale of these values and whether they need to be adjusted to match the expected 0-1 range.

import mne
import numpy as np

# Define subjects and their raw file paths
selected_subjects = ['02', '04', '05', '10', '17', '19', '21']

raw_file_paths = {
    '02': 'C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-02\\eeg\\sub-02_task-classicalMusic_eeg.edf',
    '04': 'C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-04\\eeg\\sub-04_task-classicalMusic_eeg.edf',
    '05': 'C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-05\\eeg\\sub-05_task-classicalMusic_eeg.edf',
    '10': 'C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-10\\eeg\\sub-10_task-classicalMusic_eeg.edf',
    '17': 'C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-17\\eeg\\sub-17_task-classicalMusic_eeg.edf',
    '19': 'C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-19\\eeg\\sub-19_task-classicalMusic_eeg.edf',
    '21': 'C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-21\\eeg\\sub-21_task-classicalMusic_eeg.edf'
}


# Initialize variables to track the min and max values
valance_min, valance_max = np.inf, -np.inf
arousal_min, arousal_max = np.inf, -np.inf

# Loop over each subject
for subject in selected_subjects:
    # Load the raw data
    raw_file_path = raw_file_paths[subject]
    raw = mne.io.read_raw_edf(raw_file_path, preload=True)
    
    # Extract the 'ft_valance' and 'ft_arousal' data
    ft_valance_data = raw.copy().pick_channels(['ft_valance']).get_data()
    ft_arousal_data = raw.copy().pick_channels(['ft_arousal']).get_data()
    
    # Update the min and max values if necessary
    valance_min = min(valance_min, ft_valance_data.min())
    valance_max = max(valance_max, ft_valance_data.max())
    arousal_min = min(arousal_min, ft_arousal_data.min())
    arousal_max = max(arousal_max, ft_arousal_data.max())

# Print out the min and max values
print(f"Valance range: {valance_min} to {valance_max}")
print(f"Arousal range: {arousal_min} to {arousal_max}")

In [ ]:
# read the EDF file header 

import mne

# Choose a subject and its raw file path for demonstration
subject = '02'
raw_file_path = raw_file_paths[subject]

# Read the EDF file
raw = mne.io.read_raw_edf(raw_file_path, preload=False, verbose=False)

# Display the info attribute of the Raw object
print(raw.info)


In [ ]:
# Extract specific header details for channels
for ch in raw.info['chs']:
    print(f"Channel: {ch['ch_name']}")
    print(f"  Scaling factor: {ch['cal']}")
    print(f"  Physical range: {ch['range']}")
    print(f"  Channel type: {ch['kind']}\n")

In [ ]:
# 2)-3) 
# since scaling value of 1.0, use the following function rescale to the article's range of -5 to 5

import mne
import numpy as np

# Define subjects and their raw file paths
selected_subjects = ['02', '04', '05', '10', '17', '19', '21']

raw_file_paths = {
    '02': 'C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-02\\eeg\\sub-02_task-classicalMusic_eeg.edf',
    '04': 'C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-04\\eeg\\sub-04_task-classicalMusic_eeg.edf',
    '05': 'C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-05\\eeg\\sub-05_task-classicalMusic_eeg.edf',
    '10': 'C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-10\\eeg\\sub-10_task-classicalMusic_eeg.edf',
    '17': 'C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-17\\eeg\\sub-17_task-classicalMusic_eeg.edf',
    '19': 'C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-19\\eeg\\sub-19_task-classicalMusic_eeg.edf',
    '21': 'C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-21\\eeg\\sub-21_task-classicalMusic_eeg.edf'
}

def rescale_to_range(values, min_value, max_value):
    """
    Rescale values to a given range.
    
    Args:
    - values (array-like): The values to be rescaled.
    - min_value (float): The minimum value of the new range.
    - max_value (float): The maximum value of the new range.
    
    Returns:
    - np.array: Rescaled values.
    """
    # Normalize to 0-1 range
    normalized_values = (values - np.min(values)) / (np.max(values) - np.min(values))
    
    # Rescale to desired range
    rescaled_values = (max_value - min_value) * normalized_values + min_value
    
    return rescaled_values

# Extract and rescale valence and arousal values for each subject and trial
rescaled_data = {}

channels_of_interest = ['ft_valance', 'ft_arousal']

for subject, raw_file_path in raw_file_paths.items():
    # Load the raw data
    raw = mne.io.read_raw_edf(raw_file_path, preload=True, verbose=False)
    
    # Diagnostic check: Check channels in raw data
    print(f"Channels in raw data for subject {subject}: {raw.ch_names}")
    
    # Extract data for channels of interest
    data = raw.copy().pick(channels_of_interest).get_data()
    valance_data = data[0]
    arousal_data = data[1]
    
    rescaled_valance = rescale_to_range(valance_data, -5, 5)
    rescaled_arousal = rescale_to_range(arousal_data, -5, 5)
    
    rescaled_data[subject] = {
        'valance': rescaled_valance,
        'arousal': rescaled_arousal
    }

print(rescaled_data['02']['valance'][:10], rescaled_data['02']['arousal'][:10])  # Displaying first 10 values for subject 02 as an example

above: success! now, use in below code for visual inspections before DL, downstream analyses

In [ ]:
feeltrace_values


In [ ]:
# Define subjects and their raw file paths
selected_subjects = ['02', '04', '05', '10', '17', '19', '21']

raw_file_paths = {
    '02': 'C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-02\\eeg\\sub-02_task-classicalMusic_eeg.edf',
    '04': 'C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-04\\eeg\\sub-04_task-classicalMusic_eeg.edf',
    '05': 'C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-05\\eeg\\sub-05_task-classicalMusic_eeg.edf',
    '10': 'C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-10\\eeg\\sub-10_task-classicalMusic_eeg.edf',
    '17': 'C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-17\\eeg\\sub-17_task-classicalMusic_eeg.edf',
    '19': 'C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-19\\eeg\\sub-19_task-classicalMusic_eeg.edf',
    '21': 'C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-21\\eeg\\sub-21_task-classicalMusic_eeg.edf'
}
events_dict = {
    '02': [[start_trial_1_sub02, 0, 1], [end_trial_1_sub02, 0, 2], [start_trial_2_sub02, 0, 1], [end_trial_2_sub02, 0, 2]],
    '04': [[start_trial_2_sub04, 0, 1], [end_trial_2_sub04, 0, 2], [start_trial_3_sub04, 0, 1], [end_trial_3_sub04, 0, 2], [start_trial_5_sub04, 0, 1], [end_trial_5_sub04, 0, 2], [start_trial_8_sub04, 0, 1], [end_trial_8_sub04, 0, 2]],
    '05': [[start_trial_1_sub05, 0, 1], [end_trial_1_sub05, 0, 2], [start_trial_3_sub05, 0, 1], [end_trial_3_sub05, 0, 2], [start_trial_7_sub05, 0, 1], [end_trial_7_sub05, 0, 2], [start_trial_9_sub05, 0, 1], [end_trial_9_sub05, 0, 2]],
    '10': [[start_trial_1_sub10, 0, 1], [end_trial_1_sub10, 0, 2], [start_trial_4_sub10, 0, 1], [end_trial_4_sub10, 0, 2], [start_trial_6_sub10, 0, 1], [end_trial_6_sub10, 0, 2], [start_trial_8_sub10, 0, 1], [end_trial_8_sub10, 0, 2]],
    '17': [[start_trial_1_sub17, 0, 1], [end_trial_1_sub17, 0, 2], [start_trial_2_sub17, 0, 1], [end_trial_2_sub17, 0, 2], [start_trial_5_sub17, 0, 1], [end_trial_5_sub17, 0, 2], [start_trial_6_sub17, 0, 1], [end_trial_6_sub17, 0, 2]],
    '19': [[start_trial_2_sub19, 0, 1], [end_trial_2_sub19, 0, 2], [start_trial_4_sub19, 0, 1], [end_trial_4_sub19, 0, 2], [start_trial_5_sub19, 0, 1], [end_trial_5_sub19, 0, 2], [start_trial_8_sub19, 0, 1], [end_trial_8_sub19, 0, 2]],
    '21': [[start_trial_2_sub21, 0, 1], [end_trial_2_sub21, 0, 2], [start_trial_4_sub21, 0, 1], [end_trial_4_sub21, 0, 2], [start_trial_5_sub21, 0, 1], [end_trial_5_sub21, 0, 2], [start_trial_8_sub21, 0, 1], [end_trial_8_sub21, 0, 2]],
}

channels_of_interest = ['ft_valance', 'ft_arousal', 'music']

# Rescaling function
def rescale_to_range(data, new_min, new_max):
    old_min, old_max = np.min(data), np.max(data)
    return new_min + (data - old_min) * (new_max - new_min) / (old_max - old_min)

for subject in selected_subjects:
    # Load the raw data
    raw_file_path = raw_file_paths[subject]
    raw = mne.io.read_raw_edf(raw_file_path, preload=True)
    
    # Rescale channels of interest
    for ch in channels_of_interest:
        if ch in raw.ch_names:
            data = raw.copy().pick(ch).get_data()[0]
            if ch == 'music':
                rescaled_data = rescale_to_range(data, 0, 4)
            else:
                rescaled_data = rescale_to_range(data, -5, 5)
            raw._data[raw.ch_names.index(ch)] = rescaled_data
    
    # Diagnostic check 1: Check channels in raw data
    print(f"Channels in raw data for subject {subject}: {raw.ch_names}")
    
    # Extract events for the current subject from the events_dict
    events = events_dict[subject]
    trial_starts = [event[0] for event in events if event[2] == 1]
    trial_ends = [event[0] for event in events if event[2] == 2]
    tmax_values = [(end - start) / raw.info['sfreq'] for start, end in zip(trial_starts, trial_ends)]
    
    feeltrace_values = []
    
    for i, start in enumerate(trial_starts):
        musicandreporting_trialsegs = mne.Epochs(raw, events=[events[i*2]], event_id={'start': 1}, tmin=0, tmax=tmax_values[i], baseline=None, preload=True)
        
        # Extract FEELTRACE values for the segment
        valance_data = musicandreporting_trialsegs.get_data(picks='ft_valance')[0]
        arousal_data = musicandreporting_trialsegs.get_data(picks='ft_arousal')[0]
    
        # Append the time-series values to our list
        feeltrace_values.append([valance_data, arousal_data])
        
        # Diagnostic check 2: Check channels in epochs_temp
        print(f"Channels in musicandreporting_trialsegs for subject {subject}, trial {i+1}: {musicandreporting_trialsegs.ch_names}")
        
        for ch in channels_of_interest:
            if ch in musicandreporting_trialsegs.ch_names:
                data = musicandreporting_trialsegs.get_data(picks=ch)[0]
                times = musicandreporting_trialsegs.times
                                
                # Plot
                plt.figure(figsize=(15, 4))
                plt.plot(times, data[-1, :])  
                plt.title(f"Subject: {subject} | Channel: {ch} | Trial {i+1}")
                plt.xlabel('Time (s)')
                plt.ylabel('Value')
                
                # Set x-axis ticks for every 10 seconds
                x_ticks = np.arange(times[0], times[-1], 10)
                plt.xticks(x_ticks, [f"{tick:.2f}" for tick in x_ticks])
                plt.grid(True)
                plt.tight_layout()
                plt.show()

# section 6: epoching for musical phrases, then musical moments

# section 6 sample numbers for phrases and moments

In [ ]:
# subject 2's sample numbers for musical phrases and moments

edf_file_path = 'C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-02\\eeg\\sub-02_task-classicalMusic_eeg.edf'  
raw = mne.io.read_raw_edf(edf_file_path, preload=True)
sfreq = raw.info['sfreq']  # get the sampling frequency


# pieces
start_piece3_sub02 = int(17.092 * sfreq)
end_piece3_sub02= int(167.092 * sfreq)

start_piece2_sub02 = int(175.484 * sfreq)
end_piece2_sub02= int(302.484 * sfreq)

# phrases
start_phrase1_sub02_p3 = int(125.092 * sfreq)
end_phrase1_sub02_p3= int(147.092 * sfreq)

start_phrase2_sub02_p2 = int(175.484 * sfreq)
end_phrase2_sub02_p2= int(197.484 * sfreq)

# moment within each phrase, piece numbers correpsond
#start_moment_1_sub02_p3 = int(128.092 * sfreq)
#end_moment_1_sub02_p3 = int(129.092 * sfreq)

#start_moment_2_sub02_p2 = int(194.484 * sfreq)
#end_moment_2_sub02_p2 = int(195.484 * sfreq)

In [ ]:
# subject 4's sample numbers for musical phrases and moments

edf_file_path = 'C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-04\\eeg\\sub-04_task-classicalMusic_eeg.edf'  
raw = mne.io.read_raw_edf(edf_file_path, preload=True)
sfreq = raw.info['sfreq']  # get the sampling frequency

# pieces
start_piece3_sub04 = int(605.756 * sfreq)
end_piece3_sub04= int(755.756 * sfreq)

start_piece2_sub04 = int(337.511 * sfreq)
end_piece2_sub04= int(464.511 * sfreq)

start_piece5_sub04 = int(173.555 * sfreq)
end_piece5_sub04= int(333.555 * sfreq)

# phrases
start_phrase1_sub04_p3 = int(713.756 * sfreq)
end_phrase1_sub04_p3= int(735.756 * sfreq)

start_phrase2_sub04_p2 = int(337.511 * sfreq)
end_phrase2_sub04_p2= int(359.511 * sfreq)

start_phrase3_sub04_p5 = int(213.555 * sfreq)
end_phrase3_sub04_p5= int(228.555 * sfreq)

# moment within each phrase, piece numbers correpsond
#start_moment_1_sub04_p3 = int(716.756 * sfreq)
#end_moment_1_sub04_p3 = int(717.756 * sfreq)

#start_moment_2_sub04_p2 = int(356.511 * sfreq)
#end_moment_2_sub04_p2 = int(357.511 * sfreq)

#start_moment_3_sub04_p5 = int(227.555 * sfreq)
#end_moment_3_sub04_p5 = int(228.555 * sfreq)

In [ ]:
# subject 5's sample numbers for musical phrases and moments

edf_file_path = 'C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-05\\eeg\\sub-05_task-classicalMusic_eeg.edf'  
raw = mne.io.read_raw_edf(edf_file_path, preload=True)
sfreq = raw.info['sfreq']  # get the sampling frequency

# pieces
start_piece3_sub05 = int(908.811 * sfreq)
end_piece3_sub05= int(1058.811 * sfreq)

start_piece2_sub05 = int(14.594 * sfreq)
end_piece2_sub05= int(141.594 * sfreq)

start_piece5_sub05 = int(282.772 * sfreq)
end_piece5_sub05= int(442.772 * sfreq)

# phrases
start_phrase1_sub05_p3 = int(1016.811 * sfreq)
end_phrase1_sub05_p3= int(1038.811 * sfreq)

start_phrase2_sub05_p2 = int(14.594 * sfreq)
end_phrase2_sub05_p2= int(36.594 * sfreq)

start_phrase3_sub05_p5 = int(322.722 * sfreq)
end_phrase3_sub05_p5= int(337.772 * sfreq)

# moment within each phrase, piece numbers correpsond
#start_moment_1_sub05_p3 = int(1019.811 * sfreq)
#end_moment_1_sub05_p3 = int(1020.811 * sfreq)

#start_moment_2_sub05_p2 = int(33.594 * sfreq)
#end_moment_2_sub05_p2 = int(34.594 * sfreq)

#start_moment_3_sub05_p5 = int(336.772 * sfreq)
#end_moment_3_sub05_p5 = int(337.772 * sfreq)

In [ ]:
# subject 10's sample numbers for musical phrases and moments

edf_file_path = 'C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-10\\eeg\\sub-10_task-classicalMusic_eeg.edf'  
raw = mne.io.read_raw_edf(edf_file_path, preload=True)
sfreq = raw.info['sfreq']  # get the sampling frequency

# pieces
start_piece3_sub10 = int(447.09 * sfreq)
end_piece3_sub10= int(597.09 * sfreq)

start_piece2_sub10 = int(14.906 * sfreq)
end_piece2_sub10= int(141.906 * sfreq)

start_piece5_sub10 = int(1089.656 * sfreq)
end_piece5_sub10= int(1249.656 * sfreq)

# phrases
start_phrase1_sub10_p3 = int(555.09 * sfreq)
end_phrase1_sub10_p3= int(577.09 * sfreq)

start_phrase2_sub10_p2 = int(14.906 * sfreq)
end_phrase2_sub10_p2= int(36.906 * sfreq)

start_phrase3_sub10_p5 = int(1129.656 * sfreq)
end_phrase3_sub10_p5= int(1144.656 * sfreq)

# moment within each phrase, piece numbers correpsond
#start_moment_1_sub10_p3 = int(558.09 * sfreq)
#end_moment_1_sub10_p3 = int(559.09 * sfreq)

#start_moment_2_sub10_p2 = int(33.906 * sfreq)
#end_moment_2_sub10_p2 = int(34.906 * sfreq)

#start_moment_3_sub10_p5 = int(1143.656 * sfreq)
#end_moment_3_sub10_p5 = int(1144.656 * sfreq)

In [ ]:
# subject 17's sample numbers for musical phrases and moments

edf_file_path = 'C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-17\\eeg\\sub-17_task-classicalMusic_eeg.edf'  
raw = mne.io.read_raw_edf(edf_file_path, preload=True)
sfreq = raw.info['sfreq']  # get the sampling frequency

# pieces
start_piece3_sub17 = int(14.109 * sfreq)
end_piece3_sub17= int(164.109 * sfreq)

start_piece2_sub17 = int(791.817 * sfreq)
end_piece2_sub17= int(918.817 * sfreq)

start_piece5_sub17 = int(628.262 * sfreq)
end_piece5_sub17= int(788.262 * sfreq)

# phrases
start_phrase1_sub17_p3 = int(122.109 * sfreq)
end_phrase1_sub17_p3= int(144.109 * sfreq)

start_phrase2_sub17_p2 = int(791.817 * sfreq)
end_phrase2_sub17_p2= int(813.817 * sfreq)

start_phrase3_sub17_p5 = int(668.262 * sfreq)
end_phrase3_sub17_p5= int(683.262 * sfreq)

# moment within each phrase, piece numbers correpsond
#start_moment_1_sub17_p3 = int(125.109 * sfreq)
#end_moment_1_sub17_p3 = int(126.109 * sfreq)

#start_moment_2_sub17_p2 = int(810.817 * sfreq)
#end_moment_2_sub17_p2 = int(811.817 * sfreq)

#start_moment_3_sub17_p5 = int(682.262 * sfreq)
#end_moment_3_sub17_p5 = int(683.262 * sfreq)

In [ ]:
# subject 19's sample numbers for musical phrases and moments

edf_file_path = 'C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-19\\eeg\\sub-19_task-classicalMusic_eeg.edf'  
raw = mne.io.read_raw_edf(edf_file_path, preload=True)
sfreq = raw.info['sfreq']  # get the sampling frequency

# pieces
start_piece3_sub19 = int(637.032 * sfreq)
end_piece3_sub19= int(787.032 * sfreq)

start_piece2_sub19 = int(502.826 * sfreq)
end_piece2_sub19= int(629.826 * sfreq)

start_piece5_sub19 = int(1124.542 * sfreq)
end_piece5_sub19= int(1284.542 * sfreq)

# phrases
start_phrase1_sub19_p3 = int(745.032 * sfreq)
end_phrase1_sub19_p3= int(767.032 * sfreq)

start_phrase2_sub19_p2 = int(502.826 * sfreq)
end_phrase2_sub19_p2= int(524.826 * sfreq)

start_phrase3_sub19_p5 = int(1161.608 * sfreq)
end_phrase3_sub19_p5= int(1176.608 * sfreq)

# moment within each phrase, piece numbers correpsond
#start_moment_1_sub19_p3 = int(748.032 * sfreq)
#end_moment_1_sub19_p3 = int(749.032 * sfreq)

#start_moment_2_sub19_p2 = int(521.826 * sfreq)
#end_moment_2_sub19_p2 = int(522.826 * sfreq)

#start_moment_3_sub19_p5 = int(1175.608 * sfreq)
#end_moment_3_sub19_p5 = int(1176.608 * sfreq)

In [ ]:
# subject 21's sample numbers for musical phrases and moments

edf_file_path = 'C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-21\\eeg\\sub-21_task-classicalMusic_eeg.edf'  
raw = mne.io.read_raw_edf(edf_file_path, preload=True)
sfreq = raw.info['sfreq']  # get the sampling frequency

# pieces
start_piece3_sub21 = int(636.067 * sfreq)
end_piece3_sub21= int(786.067 * sfreq)

start_piece2_sub21 = int(502.01 * sfreq)
end_piece2_sub21= int(629.01 * sfreq)

start_piece5_sub21 = int(1121.608 * sfreq)
end_piece5_sub21= int(1281.608 * sfreq)

# phrases
start_phrase1_sub21_p3 = int(744.067 * sfreq)
end_phrase1_sub21_p3= int(766.067 * sfreq)

start_phrase2_sub21_p2 = int(502.01 * sfreq)
end_phrase2_sub21_p2= int(524.01 * sfreq)

start_phrase3_sub21_p5 = int(1161.608 * sfreq)
end_phrase3_sub21_p5= int(1176.608 * sfreq)

# moment within each phrase, piece numbers correpsond
#start_moment_1_sub21_p3 = int(747.067 * sfreq)
#end_moment_1_sub21_p3 = int(748.067 * sfreq)

#start_moment_2_sub21_p2 = int(521.01 * sfreq)
#end_moment_2_sub21_p2 = int(524.01 * sfreq)

#start_moment_3_sub21_p5 = int(1175.608 * sfreq)
#end_moment_3_sub21_p5 = int(1176.608 * sfreq)

# section 6 epoching/segmenting for musical phrases, moments

In [ ]:
# for EEG data epoched/segmented into pieces & phrases, respectively
# USE processed-for-analysis AS PATH!

import os
import mne

# Generic function to save epochs for musical segments (phrases or moments)
def save_epochs_for_segments(subject, start_end_samples, raw, output_folder, segment_type):
    """Saves epochs for each musical segment (phrase or moment) based on start and end samples."""
    subject_folder = os.path.join(output_folder, subject)
    if not os.path.exists(subject_folder):
        os.makedirs(subject_folder)
    
    for i, (start, end) in enumerate(start_end_samples):
        epochs = mne.Epochs(raw, events=[[start, 0, 1], [end, 0, 2]], event_id={segment_type: 1},
                            tmin=0, tmax=(end - start) / raw.info['sfreq'], baseline=None, preload=True)
        epochs.save(os.path.join(subject_folder, f"{subject}_musical{segment_type}_{i+1}-epo.fif"), overwrite=True)


# use the processed-for-analysis, not raw, for this 'raw_file_paths' variable
raw_file_paths = {
    '02': 'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\processed-for-analysis\\sub-02_task-classicalMusic_eeg_raw-with-ICA-weights-filtered-and-referenced_analysis.fif',
    '04': 'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\processed-for-analysis\\sub-04_task-classicalMusic_eeg_raw-with-ICA-weights-filtered-and-referenced_analysis.fif',
    '05': 'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\processed-for-analysis\\sub-05_task-classicalMusic_eeg_raw-with-ICA-weights-filtered-and-referenced_analysis.fif',
    '10': 'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\processed-for-analysis\\sub-10_task-classicalMusic_eeg_raw-with-ICA-weights-filtered-and-referenced_analysis.fif',
    '17': 'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\processed-for-analysis\\sub-17_task-classicalMusic_eeg_raw-with-ICA-weights-filtered-and-referenced_analysis.fif',
    '19': 'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\processed-for-analysis\\sub-19_task-classicalMusic_eeg_raw-with-ICA-weights-filtered-and-referenced_analysis.fif',
    '21': 'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\processed-for-analysis\\sub-21_task-classicalMusic_eeg_raw-with-ICA-weights-filtered-and-referenced_analysis.fif'
}

# Dictionaries for musical phrases and moments
musical_pieces_samples = {
    '02': [(start_piece3_sub02, end_piece3_sub02), (start_piece2_sub02, end_piece2_sub02)],
    '04': [(start_piece3_sub04, end_piece3_sub04), (start_piece2_sub04, end_piece2_sub04), (start_piece5_sub04, end_piece5_sub04)],
    '05': [(start_piece3_sub05, end_piece3_sub05), (start_piece2_sub05, end_piece2_sub05), (start_piece5_sub05, end_piece5_sub05)],
    '10': [(start_piece3_sub10, end_piece3_sub10), (start_piece2_sub10, end_piece2_sub10), (start_piece5_sub10, end_piece5_sub10)],
    '17': [(start_piece3_sub17, end_piece3_sub17), (start_piece2_sub17, end_piece2_sub17), (start_piece5_sub17, end_piece5_sub17)],
    '19': [(start_piece3_sub19, end_piece3_sub19), (start_piece2_sub19, end_piece2_sub19), (start_piece5_sub19, end_piece5_sub19)], 
    '21': [(start_piece3_sub21, end_piece3_sub21), (start_piece2_sub21, end_piece2_sub21), (start_piece5_sub21, end_piece5_sub21)]
}
musical_phrases_samples = {
    '02': [(start_phrase1_sub02_p3, end_phrase1_sub02_p3), (start_phrase2_sub02_p2, end_phrase2_sub02_p2)],
    '04': [(start_phrase1_sub04_p3, end_phrase1_sub04_p3), (start_phrase2_sub04_p2, end_phrase2_sub04_p2), (start_phrase3_sub04_p5, end_phrase3_sub04_p5)],
    '05': [(start_phrase1_sub05_p3, end_phrase1_sub05_p3), (start_phrase2_sub05_p2, end_phrase2_sub05_p2), (start_phrase3_sub05_p5, end_phrase3_sub05_p5)],
    '10': [(start_phrase1_sub10_p3, end_phrase1_sub10_p3), (start_phrase2_sub10_p2, end_phrase2_sub10_p2), (start_phrase3_sub10_p5, end_phrase3_sub10_p5)],
    '17': [(start_phrase1_sub17_p3, end_phrase1_sub17_p3), (start_phrase2_sub17_p2, end_phrase2_sub17_p2), (start_phrase3_sub17_p5, end_phrase3_sub17_p5)],
    '19': [(start_phrase1_sub19_p3, end_phrase1_sub19_p3), (start_phrase2_sub19_p2, end_phrase2_sub19_p2), (start_phrase3_sub19_p5, end_phrase3_sub19_p5)], 
    '21': [(start_phrase1_sub21_p3, end_phrase1_sub21_p3), (start_phrase2_sub21_p2, end_phrase2_sub21_p2), (start_phrase3_sub21_p5, end_phrase3_sub21_p5)]
}

#musical_moments_samples = {
    #'02': [(start_moment_1_sub02_p3, end_moment_1_sub02_p3), (start_moment_2_sub02_p2, end_moment_2_sub02_p2)],
    #'04': [(start_moment_1_sub04_p3, end_moment_1_sub04_p3), (start_moment_2_sub04_p2, end_moment_2_sub04_p2), (start_moment_3_sub04_p5, end_moment_3_sub04_p5)],
    #'05': [(start_moment_1_sub05_p3, end_moment_1_sub05_p3), (start_moment_2_sub05_p2, end_moment_2_sub05_p2), (start_moment_3_sub05_p5, end_moment_3_sub05_p5)],
    #'10': [(start_moment_1_sub10_p3, end_moment_1_sub10_p3), (start_moment_2_sub10_p2, end_moment_2_sub10_p2), (start_moment_3_sub10_p5, end_moment_3_sub10_p5)],
    #'17': [(start_moment_1_sub17_p3, end_moment_1_sub17_p3), (start_moment_2_sub17_p2, end_moment_2_sub17_p2), (start_moment_3_sub17_p5, end_moment_3_sub17_p5)],
    #'19': [(start_moment_1_sub19_p3, end_moment_1_sub19_p3), (start_moment_2_sub19_p2, end_moment_2_sub19_p2), (start_moment_3_sub19_p5, end_moment_3_sub19_p5)], 
    #'21': [(start_moment_1_sub21_p3, end_moment_1_sub21_p3), (start_moment_2_sub21_p2, end_moment_2_sub21_p2), (start_moment_3_sub21_p5, end_moment_3_sub21_p5)]
#}

# Folder structure
musicalpieces_output_folder = 'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data'
# Create the musical moments output folder if it doesn't exist
if not os.path.exists(musicalmoments_output_folder):
    os.makedirs(musicalmoments_output_folder, exist_ok=True)

musicalphrases_output_folder = 'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data'
# Create the musical phrases output folder if it doesn't exist
if not os.path.exists(musicalphrases_output_folder):
    os.makedirs(musicalphrases_output_folder, exist_ok=True)

# Load the processed data and save epochs for phrases and moments
for subject, start_end_samples in musical_pieces_samples.items():
    processed_raw = mne.io.read_raw_fif(raw_file_paths[subject], preload=True)
    save_epochs_for_segments(subject, start_end_samples, processed_raw, musicalpieces_output_folder, 'piece')

for subject, start_end_samples in musical_phrases_samples.items():
    processed_raw = mne.io.read_raw_fif(raw_file_paths[subject], preload=True)
    save_epochs_for_segments(subject, start_end_samples, processed_raw, musicalphrases_output_folder, 'phrase')


In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler

# Let's say you have 100 epochs, 32 channels, and 1000 timepoints per epoch
# This is your three-dimensional EEG data array:
eeg_data = np.random.rand(100, 32, 1000)  # Replace with your actual EEG data

# Reshape the data to two dimensions: (n_samples, n_features)
# This flattens the last two dimensions
eeg_data_reshaped = eeg_data.reshape(100, -1)  # Now of shape (100, 32000)

# It's often a good idea to scale the features
scaler = StandardScaler()
eeg_data_scaled = scaler.fit_transform(eeg_data_reshaped)

# Now you can use eeg_data_scaled with UMAP

In [ ]:
from sklearn.manifold import TSNE, LocallyLinearEmbedding, Isomap
import matplotlib.pyplot as plt

# t-SNE
tsne = TSNE(n_components=2, random_state=42)
eeg_tsne = tsne.fit_transform(eeg_data_scaled)

# LLE
lle = LocallyLinearEmbedding(n_components=2, random_state=42)
eeg_lle = lle.fit_transform(eeg_data_scaled)

# Isomap
isomap = Isomap(n_components=2)
eeg_isomap = isomap.fit_transform(eeg_data_scaled)


In [ ]:
# for the piece and phrase visualizations, respectively
# USE RAW EEG FILE AS PATH

import matplotlib.pyplot as plt
import mne
import numpy as np

# Define paths to the processed epochs and the raw EEG files
processed_epochs_paths = {
    '02': [
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\02\\02_musicalphrase_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\02\\02_musicalphrase_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\02\\02_musicalpiece_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\02\\02_musicalpiece_2-epo.fif',
    ],
    '04': [
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\04\\04_musicalphrase_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\04\\04_musicalphrase_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\04\\04_musicalphrase_3-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\04\\04_musicalpiece_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\04\\04_musicalpiece_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\04\\04_musicalpiece_3-epo.fif'
    ],
    '05': [
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\05\\05_musicalphrase_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\05\\05_musicalphrase_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\05\\05_musicalphrase_3-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\05\\05_musicalpiece_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\05\\05_musicalpiece_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\05\\05_musicalpiece_3-epo.fif'
    ],
    '10': [
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\10\\10_musicalphrase_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\10\\10_musicalphrase_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\10\\10_musicalphrase_3-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\10\\10_musicalpiece_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\10\\10_musicalpiece_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\10\\10_musicalpiece_3-epo.fif'
    ],
    '17': [
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\17\\17_musicalphrase_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\17\\17_musicalphrase_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\17\\17_musicalphrase_3-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\17\\17_musicalpiece_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\17\\17_musicalpiece_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\17\\17_musicalpiece_3-epo.fif'
    ],
    '19': [
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\19\\19_musicalphrase_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\19\\19_musicalphrase_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\19\\19_musicalphrase_3-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\19\\19_musicalpiece_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\19\\19_musicalpiece_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\19\\19_musicalpiece_3-epo.fif'
    ],
    '21': [
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\21\\21_musicalphrase_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\21\\21_musicalphrase_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\21\\21_musicalphrase_3-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\21\\21_musicalpiece_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\21\\21_musicalpiece_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\21\\21_musicalpiece_3-epo.fif'
    ],
}

raw_file_paths = {
    '02': 'C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-02\\eeg\\sub-02_task-classicalMusic_eeg.edf',
    '04': 'C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-04\\eeg\\sub-04_task-classicalMusic_eeg.edf',
    '05': 'C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-05\\eeg\\sub-05_task-classicalMusic_eeg.edf',
    '10': 'C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-10\\eeg\\sub-10_task-classicalMusic_eeg.edf',
    '17': 'C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-17\\eeg\\sub-17_task-classicalMusic_eeg.edf',
    '19': 'C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-19\\eeg\\sub-19_task-classicalMusic_eeg.edf',
    '21': 'C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-21\\eeg\\sub-21_task-classicalMusic_eeg.edf'
}

# Define the output folder for the figures
figures_output_folder = 'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\valencearousal_epochs'
# Create the musical phrases output folder if it doesn't exist
if not os.path.exists(musicalphrases_output_folder):
    os.makedirs(musicalphrases_output_folder, exist_ok=True)

    # Function to plot 'ft_valance' and 'ft_arousal' for a given raw segment
    def plot_feeltrace_for_segment(raw, start, end, subject, epoch_type, epoch_index):
        # Create a segment of the raw data
        raw_segment = raw.copy().crop(tmin=start, tmax=end)
        
        # Plot 'ft_valance' and 'ft_arousal'
        for ch in ['ft_valance', 'ft_arousal']:
            if ch in raw_segment.ch_names:
                data, times = raw_segment.copy().pick_channels([ch]).get_data(return_times=True)
                
                # Plot
                plt.figure(figsize=(15, 4))
                plt.plot(times, data[0, :], label=ch)
                plt.title(f"Subject: {subject} | Channel: {ch} | {epoch_type} {epoch_index}")
                plt.xlabel('Time (s)')
                plt.ylabel('Value')
                plt.legend()
                plt.grid(True)
                plt.tight_layout()
                
                # Save the figure
                plt.savefig(f"{figures_output_folder}/Subject_{subject}_{epoch_type}_{epoch_index}_{ch}.png")
                plt.close()  # Close the figure to free memory

# Define the rescaling function
def rescale_to_range(data, new_min, new_max):
    old_min, old_max = np.min(data), np.max(data)
    return new_min + (data - old_min) * (new_max - new_min) / (old_max - old_min)

# Loop over each subject and their processed epochs
for subject, epoch_paths in processed_epochs_paths.items():
    for epoch_index, epoch_path in enumerate(epoch_paths):
        try:
            # Load the processed epochs
            epochs = mne.read_epochs(epoch_path, preload=True)
            
            # Load the raw EEG data
            raw = mne.io.read_raw_edf(raw_file_paths[subject], preload=True)
            
            # Rescale channels of interest
            for ch in ['ft_valance', 'ft_arousal']:
                if ch in raw.ch_names:
                    data = raw.copy().pick(ch).get_data()[0]
                    rescaled_data = rescale_to_range(data, -5, 5)
                    raw._data[raw.ch_names.index(ch)] = rescaled_data
            
            # Define start and end times based on epochs events
            events = epochs.events
            sfreq = raw.info['sfreq']
            start_times = events[:, 0] / sfreq
            end_times = (events[:, 0] + epochs.times[-1] * sfreq) / sfreq
            
            # Loop over each epoch's start and end times to plot 'ft_valance' and 'ft_arousal'
            for start, end in zip(start_times, end_times):
                # Plot 'ft_valance' and 'ft_arousal' for this segment
                plot_feeltrace_for_segment(raw, start, end, subject, 'phrase' if 'phrase' in epoch_path else 'piece', epoch_index + 1)
                
        except OSError as e:
            print(f"An error occurred while reading {epoch_path}: {e}")

In [ ]:
import os
import mne
import numpy as np
import matplotlib.pyplot as plt
from umap import UMAP
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

processed_epochs_paths = {
    '02': [
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\02\\02_musicalphrase_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\02\\02_musicalphrase_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\02\\02_musicalpiece_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\02\\02_musicalpiece_2-epo.fif',
    ],
    '04': [
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\04\\04_musicalphrase_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\04\\04_musicalphrase_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\04\\04_musicalphrase_3-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\04\\04_musicalpiece_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\04\\04_musicalpiece_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\04\\04_musicalpiece_3-epo.fif'
    ],
    '05': [
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\05\\05_musicalphrase_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\05\\05_musicalphrase_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\05\\05_musicalphrase_3-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\05\\05_musicalpiece_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\05\\05_musicalpiece_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\05\\05_musicalpiece_3-epo.fif'
    ],
    '10': [
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\10\\10_musicalphrase_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\10\\10_musicalphrase_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\10\\10_musicalphrase_3-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\10\\10_musicalpiece_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\10\\10_musicalpiece_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\10\\10_musicalpiece_3-epo.fif'
    ],
    '17': [
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\17\\17_musicalphrase_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\17\\17_musicalphrase_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\17\\17_musicalphrase_3-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\17\\17_musicalpiece_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\17\\17_musicalpiece_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\17\\17_musicalpiece_3-epo.fif'
    ],
    '19': [
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\19\\19_musicalphrase_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\19\\19_musicalphrase_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\19\\19_musicalphrase_3-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\19\\19_musicalpiece_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\19\\19_musicalpiece_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\19\\19_musicalpiece_3-epo.fif'
    ],
    '21': [
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\21\\21_musicalphrase_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\21\\21_musicalphrase_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\21\\21_musicalphrase_3-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\21\\21_musicalpiece_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\21\\21_musicalpiece_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\21\\21_musicalpiece_3-epo.fif'
    ],
}

raw_file_paths = {
    '02': 'C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-02\\eeg\\sub-02_task-classicalMusic_eeg.edf',
    '04': 'C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-04\\eeg\\sub-04_task-classicalMusic_eeg.edf',
    '05': 'C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-05\\eeg\\sub-05_task-classicalMusic_eeg.edf',
    '10': 'C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-10\\eeg\\sub-10_task-classicalMusic_eeg.edf',
    '17': 'C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-17\\eeg\\sub-17_task-classicalMusic_eeg.edf',
    '19': 'C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-19\\eeg\\sub-19_task-classicalMusic_eeg.edf',
    '21': 'C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-21\\eeg\\sub-21_task-classicalMusic_eeg.edf'
}

# Define the output folder for the figures
figures_output_folder = 'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\valencearousal_epochs'
# Make sure the output folder exists
if not os.path.exists(figures_output_folder):
    os.makedirs(figures_output_folder, exist_ok=True)


# Define a function to save epochs and plot FEELTRACE channels
def save_and_plot_epochs_for_phrases(subject, start_end_samples, raw, output_folder):
    """Saves epochs for each musical phrase and plots FEELTRACE channels."""
    # Create the subject's folder if it doesn't exist
    subject_folder = os.path.join(output_folder, subject)
    if not os.path.exists(subject_folder):
        os.makedirs(subject_folder)
    
    # Iterate over each start and end time to create, save epochs, and plot FEELTRACE channels
    for i, (start, end) in enumerate(start_end_samples):
        epochs = mne.Epochs(raw, events=[[start, 0, 1], [end, 0, 2]], event_id={'phrase': 1},
                            tmin=0, tmax=(end - start) / raw.info['sfreq'], baseline=None, preload=True)
        epochs.save(os.path.join(subject_folder, f"{subject}_musicalphrase_{i+1}-epo.fif"), overwrite=True)
        
        # Extract and plot FEELTRACE channels
        valance_data = raw.copy().crop(tmin=start/raw.info['sfreq'], tmax=end/raw.info['sfreq']).pick_channels(['ft_valance'])
        arousal_data = raw.copy().crop(tmin=start/raw.info['sfreq'], tmax=end/raw.info['sfreq']).pick_channels(['ft_arousal'])
        
        # Plot and save the figures
        plt.figure()
        plt.plot(valance_data.times, valance_data.get_data()[0, :], label='Valance')
        plt.plot(arousal_data.times, arousal_data.get_data()[0, :], label='Arousal')
        plt.legend()
        plt.title(f"{subject} Musical Phrase {i+1} FEELTRACE")
        plt.xlabel('Time (s)')
        plt.ylabel('FEELTRACE Score')
        plt.savefig(os.path.join(subject_folder, f"{subject}_musicalphrase_{i+1}_FEELTRACE.png"))
        plt.close()

# Define the start and end samples for musical phrases for each subject
# This dictionary will need to be filled with the correct start and end times
musical_phrases_samples = {
    '02': [(start_trial_1_sub02, end_trial_1_sub02), (start_trial_2_sub02, end_trial_2_sub02), ...], # Fill in with actual values
    # Repeat for other subjects...
}

# Define subjects and their raw file paths (update with the correct paths)
raw_file_paths = {
    '02': 'path_to_sub02_raw.fif',
    # Repeat for other subjects...
}

# Define the output folder for musical phrases
musicalphrases_output_folder = 'path_to_musicalphrases_data_folder'

# Load the raw data, save epochs, and plot FEELTRACE channels for each subject
for subject, start_end_samples in musical_phrases_samples.items():
    raw = mne.io.read_raw_fif(raw_file_paths[subject], preload=True)
    save_and_plot_epochs_for_phrases(subject, start_end_samples, raw, musicalphrases_output_folder)


# section 7: old parameters; not used in article

In [ ]:
# 20:04 version, update on the color spectrum but still keep previous results for comparing parameters

import mne
import numpy as np
from mne import Epochs, find_events
from sklearn.preprocessing import StandardScaler
from umap import UMAP
import matplotlib.pyplot as plt
from scipy.signal import welch
from scipy.stats import pearsonr
from sklearn.impute import SimpleImputer
from scipy.spatial.distance import pdist, squareform
from scipy.stats import spearmanr
import matplotlib.colors as mcolors
import re

# Define paths to the processed epochs and the raw EEG files
processed_epochs_paths = {
    '02': [
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\02\\02_musicalphrase_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\02\\02_musicalphrase_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\02\\02_musicalpiece_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\02\\02_musicalpiece_2-epo.fif',
    ],
    '04': [
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\04\\04_musicalphrase_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\04\\04_musicalphrase_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\04\\04_musicalphrase_3-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\04\\04_musicalpiece_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\04\\04_musicalpiece_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\04\\04_musicalpiece_3-epo.fif'
    ],
    '05': [
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\05\\05_musicalphrase_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\05\\05_musicalphrase_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\05\\05_musicalphrase_3-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\05\\05_musicalpiece_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\05\\05_musicalpiece_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\05\\05_musicalpiece_3-epo.fif'
    ],
    '10': [
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\10\\10_musicalphrase_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\10\\10_musicalphrase_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\10\\10_musicalphrase_3-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\10\\10_musicalpiece_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\10\\10_musicalpiece_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\10\\10_musicalpiece_3-epo.fif'
    ],
    '17': [
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\17\\17_musicalphrase_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\17\\17_musicalphrase_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\17\\17_musicalphrase_3-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\17\\17_musicalpiece_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\17\\17_musicalpiece_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\17\\17_musicalpiece_3-epo.fif'
    ],
    '19': [
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\19\\19_musicalphrase_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\19\\19_musicalphrase_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\19\\19_musicalphrase_3-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\19\\19_musicalpiece_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\19\\19_musicalpiece_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\19\\19_musicalpiece_3-epo.fif'
    ],
    '21': [
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\21\\21_musicalphrase_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\21\\21_musicalphrase_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\21\\21_musicalphrase_3-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\21\\21_musicalpiece_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\21\\21_musicalpiece_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\21\\21_musicalpiece_3-epo.fif'
    ],
}

raw_file_paths = {
    '02': 'C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-02\\eeg\\sub-02_task-classicalMusic_eeg.edf',
    '04': 'C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-04\\eeg\\sub-04_task-classicalMusic_eeg.edf',
    '05': 'C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-05\\eeg\\sub-05_task-classicalMusic_eeg.edf',
    '10': 'C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-10\\eeg\\sub-10_task-classicalMusic_eeg.edf',
    '17': 'C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-17\\eeg\\sub-17_task-classicalMusic_eeg.edf',
    '19': 'C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-19\\eeg\\sub-19_task-classicalMusic_eeg.edf',
    '21': 'C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-21\\eeg\\sub-21_task-classicalMusic_eeg.edf'
}

def create_sub_epochs(epochs, epoch_type):
    sub_epochs_data = []
    sfreq = epochs.info['sfreq']  # Sampling frequency
    
    if epoch_type == 'musicalphrase':
        sub_epoch_length = 0.1  # 100ms for phrases
    elif epoch_type == 'musicalpiece':
        sub_epoch_length = 0.5 # 500ms for pieces
    else:
        raise ValueError(f"Unknown epoch type: {epoch_type}")

    # Calculate the number of samples for the given sub_epoch_length
    sub_epoch_samples = int(sub_epoch_length * sfreq)

    # Create sub-epochs
    for start_time in np.arange(0, epochs.tmax, sub_epoch_length):
        start_idx = int(start_time * sfreq)
        end_idx = start_idx + sub_epoch_samples
        if end_idx > len(epochs.times):  # Ensure we don't go past the end
            break

        # Ensure indices are integers and correct slicing
        start_idx = int(start_idx)
        end_idx = int(end_idx)
        # Append sub-epoch data
        sub_epochs_data.append(epochs.copy().crop(tmin=epochs.times[start_idx], tmax=epochs.times[end_idx-1], include_tmax=True))
    
    return sub_epochs_data

def extract_valance_arousal_for_epoch(epoch, raw, valence_channel='ft_valance', arousal_channel='ft_arousal'):
    """
    Extracts valence and arousal data for a given epoch from the raw EDF file.
    
    :param epoch: The epoch for which to extract valence and arousal
    :param raw: The raw EDF data containing valence and arousal channels
    :param valence_channel: The name of the valence channel
    :param arousal_channel: The name of the arousal channel
    :return: A tuple containing the valence and arousal data arrays
    """
    # Get the start and end times of the epoch
    start, end = epoch.times[0], epoch.times[-1]
    start_sample, end_sample = int(start * raw.info['sfreq']), int(end * raw.info['sfreq'])
    
    # Extract valence and arousal data for the duration of the epoch
    valence_data = raw.copy().pick_channels([valence_channel]).get_data(start=start_sample, stop=end_sample)
    arousal_data = raw.copy().pick_channels([arousal_channel]).get_data(start=start_sample, stop=end_sample)
    
    return valence_data, arousal_data


def extract_features(sub_epochs_data, sfreq, epoch_type):
    features = []
    # Define the frequency bands
    bands = {'delta': (0.5, 4), 'theta': (4, 8), 'alpha': (8, 13)}
    spectral_features = []
    pcc_features = []

    # Calculate the number of samples for the given sub_epoch lengths based on the sampling frequency
    if epoch_type == 'musicalphrase':
        sub_epoch_length = 0.1  # 100ms for phrases
        num_samples = int(sub_epoch_length * sfreq)
    elif epoch_type == 'musicalpiece':
        sub_epoch_length = 0.5  # 500ms for pieces
        num_samples = int(sub_epoch_length * sfreq)
    else:
        raise ValueError(f"Unknown epoch type: {epoch_type}")

    for sub_epoch in sub_epochs_data:
        # Extract the data from the EpochsFIF object
        sub_epoch_data = sub_epoch.get_data()

        # Ensure nperseg does not exceed the number of samples in the sub-epoch
        nperseg = min(sub_epoch_data.shape[2], num_samples)  # The third dimension contains the time samples

        # Compute the PSD for the sub-epoch using Welch's method with the given nperseg
        f, Pxx = welch(sub_epoch_data.squeeze(), sfreq, nperseg=nperseg)

        # Calculate the spectral power in each band
        band_powers = []
        for band_name, (fmin, fmax) in bands.items():
            band_freq_indices = (f >= fmin) & (f <= fmax)
            if band_freq_indices.any():
                band_power = Pxx[:, band_freq_indices].mean(axis=-1)
                band_powers.append(band_power)
            else:
                band_powers.append(np.nan)  # Handle empty band

        band_powers = np.hstack(band_powers) if band_powers else np.array([np.nan, np.nan, np.nan])
        spectral_features.append(band_powers)

        # Calculate Pearson correlation coefficient between pairs of channels for the sub-epoch
        pcc = np.corrcoef(sub_epoch_data.squeeze(), rowvar=False)  # Use the numpy array here
        pcc = pcc[np.triu_indices_from(pcc, k=1)]  # Take the upper triangle of the matrix, excluding the diagonal
        pcc_features.extend(pcc)

    spectral_features = np.vstack(spectral_features)
    num_sub_epochs = len(sub_epochs_data)
    pcc_features = np.array(pcc_features).reshape(num_sub_epochs, -1)
    features = np.hstack([spectral_features, pcc_features]) if pcc_features.size else spectral_features

    # After computing features, handle NaN values
    imputer = SimpleImputer(missing_values=np.nan, strategy='median')
    features_imputed = imputer.fit_transform(features)

    return features_imputed

# Define the rescaling function
def rescale_to_range(data, new_min, new_max):
    old_min, old_max = np.min(data), np.max(data)
    return new_min + (data - old_min) * (new_max - new_min) / (old_max - old_min)

def create_vivid_cmap(base_cmap_name='Spectral', num_colors=256):
    base_cmap = plt.cm.get_cmap(base_cmap_name)
    base_colors = base_cmap(np.linspace(0, 1, num_colors))
    colors_hsv = mcolors.rgb_to_hsv(base_colors[:, :3])
    colors_hsv[:, 1] = 1.0  # max saturation
    colors_hsv[:, 2] = 1.0  # max brightness
    colors_rgb = mcolors.hsv_to_rgb(colors_hsv)
    return mcolors.ListedColormap(colors_rgb, name='vivid_' + base_cmap_name)


# Initialize the scaler
scaler = StandardScaler()

# Create a 2D colormap
num_colors = 256  # Number of colors to use in each dimension
cmap = create_vivid_cmap('Spectral', num_colors)

# Read raw data only once for each subject
raw_data = {}

for subject, raw_file_path in raw_file_paths.items():
    raw_data[subject] = mne.io.read_raw_edf(raw_file_path, preload=True)
    # Rescale channels only once
    for ch in ['ft_valance', 'ft_arousal']:
        if ch in raw_data[subject].ch_names:
            data = raw_data[subject].get_data(picks=ch)
            rescaled_data = rescale_to_range(data, -5, 5)
            raw_data[subject]._data[raw_data[subject].ch_names.index(ch), :] = rescaled_data

# Now process each epoch
for subject, epoch_paths in processed_epochs_paths.items():
    for epoch_index, epoch_path in enumerate(epoch_paths):
        epochs = mne.read_epochs(epoch_path, preload=True)
        raw = raw_data[subject]  # Use the preloaded and rescaled raw data

        sfreq = epochs.info['sfreq']  # Get the sampling frequency from the epochs

        # Determine the type of epoch based on the file name
        if "musicalphrase" in epoch_path:
            epoch_type = 'musicalphrase'
            umap_params = {'n_neighbors': 5, 'min_dist': 0.1, 'n_components': 2, 'random_state': 42, 'metric': 'manhattan'}
        elif "musicalpiece" in epoch_path:
            epoch_type = 'musicalpiece'
            umap_params = {'n_neighbors': 5, 'min_dist': 0.1, 'n_components': 2, 'random_state': 42, 'metric': 'manhattan'}
        else:
            raise ValueError("Unknown epoch type in path: " + epoch_path)

        # Determine the type of epoch and extract the number
        if "musicalphrase" in epoch_path:
            epoch_type = 'musicalphrase'
            match = re.search(r'musicalphrase_(\d+)', epoch_path)
        elif "musicalpiece" in epoch_path:
            epoch_type = 'musicalpiece'
            match = re.search(r'musicalpiece_(\d+)', epoch_path)
        else:
            raise ValueError("Unknown epoch type in path: " + epoch_path)
        
        # Extract the number (x) from the match if it exists
        number = match.group(1) if match else "unknown"


        # Use the determined epoch type to create sub-epochs
        sub_epochs_data = create_sub_epochs(epochs, epoch_type=epoch_type)

        # Extract valence and arousal data for each sub-epoch   
        valance_arousal = []
        for sub_epoch in sub_epochs_data:
            valence_data, arousal_data = extract_valence_arousal_for_epoch(sub_epoch, raw)
            valance_arousal.append(np.hstack((valence_data.mean(), arousal_data.mean())))
        valance_arousal = np.array(valance_arousal)

        # Extract features from the sub-epochs
        features = extract_features(sub_epochs_data, sfreq, epoch_type)

        # Check if features were extracted
        if features is None or len(features) == 0:
            print(f"Subject {subject} Epoch {epoch_index+1} has no valid features. Skipping.")
            continue

        # Handle NaNs before scaling
        imputer = SimpleImputer(missing_values=np.nan, strategy='median')
        features_imputed = imputer.fit_transform(features)

        # Proceed with scaling and UMAP if features are present
        features_scaled = scaler.fit_transform(features_imputed)

        # Apply UMAP with the specific parameters for each epoch type
        umap = UMAP(**umap_params)
        embedding = umap.fit_transform(features_scaled)


        # Before computing Spearman correlation, check for constant arrays
        if np.std(umap_distances.ravel()) == 0 or np.std(valence_arousal_distances.ravel()) == 0:
            # Handle the case where there's no variation in distances
            # For example:
            correlation = np.nan
            p_value = np.nan
        else:
            correlation, p_value = spearmanr(umap_distances.ravel(), valence_arousal_distances.ravel())
        # Compute Spearman correlation between UMAP and valence/arousal distances
        umap_distances = squareform(pdist(embedding, 'euclidean'))
        valence_arousal_distances = squareform(pdist(valance_arousal, 'euclidean'))
        correlation, p_value = spearmanr(umap_distances.ravel(), valence_arousal_distances.ravel())
        print(f"Spearman correlation between UMAP distances and valence/arousal distances: {correlation} (p-value: {p_value})")
        
        # Print shapes of the arrays to debug
        print(f"Subject {subject} Epoch {epoch_index+1}")
        print("embedding.shape:", embedding.shape)
        print("valance_arousal.shape:", valance_arousal.shape)
        
        # Before normalization, check if the peak-to-peak value is zero and handle it
        if np.ptp(valance_arousal[:, 0]) == 0 or np.ptp(valance_arousal[:, 1]) == 0:
            # Handle the case where there's no variation in the valence or arousal
            # Maybe set normalized values to zero or skip this epoch
            # For example:
            valence_normalized = np.zeros_like(valance_arousal[:, 0])
            arousal_normalized = np.zeros_like(valance_arousal[:, 1])
        else:
            valence_normalized = (valance_arousal[:, 0] - np.min(valance_arousal[:, 0])) / np.ptp(valance_arousal[:, 0])
            arousal_normalized = (valance_arousal[:, 1] - np.min(valance_arousal[:, 1])) / np.ptp(valance_arousal[:, 1])    

        # Compute a color index for each point
        valence_colors = np.int_(valence_normalized * (num_colors - 1))
        arousal_colors = np.int_(arousal_normalized * (num_colors - 1))
        colors = valence_colors + arousal_colors * num_colors

        # Create a figure and a set of subplots
        fig, ax = plt.subplots(figsize=(12, 10))

        # Plot UMAP embedding
        scatter = ax.scatter(embedding[:, 0], embedding[:, 1], c=colors, cmap=cmap, s=5)

        # Add colorbar using the computed 2D color mapping
        norm = plt.Normalize(vmin=0, vmax=num_colors**2 - 1)
        sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
        sm.set_array([])
        cbar = fig.colorbar(sm, ax=ax, orientation='horizontal')
        cbar.set_label('Valence and Arousal')

        # Set the title and labels with the new naming convention
        ax.set_title(f'UMAP of Subject {subject} {epoch_type} {number} with Valence and Arousal Overlays')

        # Remove the axes tick marks and labels
        ax.set_xticks([])
        ax.set_yticks([])

        # Show plot
        plt.show()

# section 7: Final parameters used in the article

In [ ]:
# 20:04 version, update on the color spectrum but still keep previous results for comparing parameters

import mne
import numpy as np
from mne import Epochs, find_events
from sklearn.preprocessing import StandardScaler
from umap import UMAP
import matplotlib.pyplot as plt
from scipy.signal import welch
from scipy.stats import pearsonr
from sklearn.impute import SimpleImputer
from scipy.spatial.distance import pdist, squareform
from scipy.stats import spearmanr
import matplotlib.colors as mcolors
import re

# Define paths to the processed epochs and the raw EEG files
processed_epochs_paths = {
    '02': [
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\02\\02_musicalphrase_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\02\\02_musicalphrase_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\02\\02_musicalpiece_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\02\\02_musicalpiece_2-epo.fif',
    ],
    '04': [
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\04\\04_musicalphrase_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\04\\04_musicalphrase_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\04\\04_musicalphrase_3-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\04\\04_musicalpiece_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\04\\04_musicalpiece_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\04\\04_musicalpiece_3-epo.fif'
    ],
    '05': [
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\05\\05_musicalphrase_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\05\\05_musicalphrase_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\05\\05_musicalphrase_3-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\05\\05_musicalpiece_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\05\\05_musicalpiece_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\05\\05_musicalpiece_3-epo.fif'
    ],
    '10': [
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\10\\10_musicalphrase_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\10\\10_musicalphrase_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\10\\10_musicalphrase_3-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\10\\10_musicalpiece_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\10\\10_musicalpiece_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\10\\10_musicalpiece_3-epo.fif'
    ],
    '17': [
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\17\\17_musicalphrase_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\17\\17_musicalphrase_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\17\\17_musicalphrase_3-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\17\\17_musicalpiece_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\17\\17_musicalpiece_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\17\\17_musicalpiece_3-epo.fif'
    ],
    '19': [
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\19\\19_musicalphrase_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\19\\19_musicalphrase_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\19\\19_musicalphrase_3-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\19\\19_musicalpiece_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\19\\19_musicalpiece_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\19\\19_musicalpiece_3-epo.fif'
    ],
    '21': [
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\21\\21_musicalphrase_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\21\\21_musicalphrase_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\21\\21_musicalphrase_3-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\21\\21_musicalpiece_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\21\\21_musicalpiece_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\21\\21_musicalpiece_3-epo.fif'
    ],
}

raw_file_paths = {
    '02': 'C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-02\\eeg\\sub-02_task-classicalMusic_eeg.edf',
    '04': 'C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-04\\eeg\\sub-04_task-classicalMusic_eeg.edf',
    '05': 'C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-05\\eeg\\sub-05_task-classicalMusic_eeg.edf',
    '10': 'C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-10\\eeg\\sub-10_task-classicalMusic_eeg.edf',
    '17': 'C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-17\\eeg\\sub-17_task-classicalMusic_eeg.edf',
    '19': 'C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-19\\eeg\\sub-19_task-classicalMusic_eeg.edf',
    '21': 'C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-21\\eeg\\sub-21_task-classicalMusic_eeg.edf'
}

def create_sub_epochs(epochs, epoch_type):
    sub_epochs_data = []
    sfreq = epochs.info['sfreq']  # Sampling frequency
    
    if epoch_type == 'musicalphrase':
        sub_epoch_length = 0.1  # 100ms for phrases
    elif epoch_type == 'musicalpiece':
        sub_epoch_length = 0.5 # 500ms for pieces
    else:
        raise ValueError(f"Unknown epoch type: {epoch_type}")

    # Calculate the number of samples for the given sub_epoch_length
    sub_epoch_samples = int(sub_epoch_length * sfreq)

    # Create sub-epochs
    for start_time in np.arange(0, epochs.tmax, sub_epoch_length):
        start_idx = int(start_time * sfreq)
        end_idx = start_idx + sub_epoch_samples
        if end_idx > len(epochs.times):  # Ensure we don't go past the end
            break

        # Ensure indices are integers and correct slicing
        start_idx = int(start_idx)
        end_idx = int(end_idx)
        # Append sub-epoch data
        sub_epochs_data.append(epochs.copy().crop(tmin=epochs.times[start_idx], tmax=epochs.times[end_idx-1], include_tmax=True))
    
    return sub_epochs_data

def extract_valance_arousal_for_epoch(epoch, raw, valence_channel='ft_valance', arousal_channel='ft_arousal'):
    """
    Extracts valence and arousal data for a given epoch from the raw EDF file.
    
    :param epoch: The epoch for which to extract valence and arousal
    :param raw: The raw EDF data containing valence and arousal channels
    :param valence_channel: The name of the valence channel
    :param arousal_channel: The name of the arousal channel
    :return: A tuple containing the valence and arousal data arrays
    """
    # Get the start and end times of the epoch
    start, end = epoch.times[0], epoch.times[-1]
    start_sample, end_sample = int(start * raw.info['sfreq']), int(end * raw.info['sfreq'])
    
    # Extract valence and arousal data for the duration of the epoch
    valence_data = raw.copy().pick_channels([valence_channel]).get_data(start=start_sample, stop=end_sample)
    arousal_data = raw.copy().pick_channels([arousal_channel]).get_data(start=start_sample, stop=end_sample)
    
    return valence_data, arousal_data


def extract_features(sub_epochs_data, sfreq, epoch_type):
    features = []
    # Define the frequency bands
    bands = {'delta': (0.5, 4), 'theta': (4, 8), 'alpha': (8, 13)}
    spectral_features = []
    pcc_features = []

    # Calculate the number of samples for the given sub_epoch lengths based on the sampling frequency
    if epoch_type == 'musicalphrase':
        sub_epoch_length = 0.1  # 100ms for phrases
        num_samples = int(sub_epoch_length * sfreq)
    elif epoch_type == 'musicalpiece':
        sub_epoch_length = 0.5  # 500ms for pieces
        num_samples = int(sub_epoch_length * sfreq)
    else:
        raise ValueError(f"Unknown epoch type: {epoch_type}")

    for sub_epoch in sub_epochs_data:
        # Extract the data from the EpochsFIF object
        sub_epoch_data = sub_epoch.get_data()

        # Ensure nperseg does not exceed the number of samples in the sub-epoch
        nperseg = min(sub_epoch_data.shape[2], num_samples)  # The third dimension contains the time samples

        # Compute the PSD for the sub-epoch using Welch's method with the given nperseg
        f, Pxx = welch(sub_epoch_data.squeeze(), sfreq, nperseg=nperseg)

        # Calculate the spectral power in each band
        band_powers = []
        for band_name, (fmin, fmax) in bands.items():
            band_freq_indices = (f >= fmin) & (f <= fmax)
            if band_freq_indices.any():
                band_power = Pxx[:, band_freq_indices].mean(axis=-1)
                band_powers.append(band_power)
            else:
                band_powers.append(np.nan)  # Handle empty band

        band_powers = np.hstack(band_powers) if band_powers else np.array([np.nan, np.nan, np.nan])
        spectral_features.append(band_powers)

        # Calculate Pearson correlation coefficient between pairs of channels for the sub-epoch
        pcc = np.corrcoef(sub_epoch_data.squeeze(), rowvar=False)  # Use the numpy array here
        pcc = pcc[np.triu_indices_from(pcc, k=1)]  # Take the upper triangle of the matrix, excluding the diagonal
        pcc_features.extend(pcc)

    spectral_features = np.vstack(spectral_features)
    num_sub_epochs = len(sub_epochs_data)
    pcc_features = np.array(pcc_features).reshape(num_sub_epochs, -1)
    features = np.hstack([spectral_features, pcc_features]) if pcc_features.size else spectral_features

    # After computing features, handle NaN values
    imputer = SimpleImputer(missing_values=np.nan, strategy='median')
    features_imputed = imputer.fit_transform(features)

    return features_imputed

# Define the rescaling function
def rescale_to_range(data, new_min, new_max):
    old_min, old_max = np.min(data), np.max(data)
    return new_min + (data - old_min) * (new_max - new_min) / (old_max - old_min)

def create_vivid_cmap(base_cmap_name='Spectral', num_colors=256):
    base_cmap = plt.cm.get_cmap(base_cmap_name)
    base_colors = base_cmap(np.linspace(0, 1, num_colors))
    colors_hsv = mcolors.rgb_to_hsv(base_colors[:, :3])
    colors_hsv[:, 1] = 1.0  # max saturation
    colors_hsv[:, 2] = 1.0  # max brightness
    colors_rgb = mcolors.hsv_to_rgb(colors_hsv)
    return mcolors.ListedColormap(colors_rgb, name='vivid_' + base_cmap_name)


# Initialize the scaler
scaler = StandardScaler()

# Create a 2D colormap
num_colors = 256  # Number of colors to use in each dimension
cmap = create_vivid_cmap('Spectral', num_colors)

# Read raw data only once for each subject
raw_data = {}

for subject, raw_file_path in raw_file_paths.items():
    raw_data[subject] = mne.io.read_raw_edf(raw_file_path, preload=True)
    # Rescale channels only once
    for ch in ['ft_valance', 'ft_arousal']:
        if ch in raw_data[subject].ch_names:
            data = raw_data[subject].get_data(picks=ch)
            rescaled_data = rescale_to_range(data, -5, 5)
            raw_data[subject]._data[raw_data[subject].ch_names.index(ch), :] = rescaled_data

# Now process each epoch
for subject, epoch_paths in processed_epochs_paths.items():
    for epoch_index, epoch_path in enumerate(epoch_paths):
        epochs = mne.read_epochs(epoch_path, preload=True)
        raw = raw_data[subject]  # Use the preloaded and rescaled raw data

        sfreq = epochs.info['sfreq']  # Get the sampling frequency from the epochs

        # Determine the type of epoch based on the file name
        if "musicalphrase" in epoch_path:
            epoch_type = 'musicalphrase'
            umap_params = {'n_neighbors': 5, 'min_dist': 0.1, 'n_components': 2, 'random_state': 42}
        elif "musicalpiece" in epoch_path:
            epoch_type = 'musicalpiece'
            umap_params = {'n_neighbors': 5, 'min_dist': 0.1, 'n_components': 2, 'random_state': 42}
        else:
            raise ValueError("Unknown epoch type in path: " + epoch_path)

        # Determine the type of epoch and extract the number
        if "musicalphrase" in epoch_path:
            epoch_type = 'musicalphrase'
            match = re.search(r'musicalphrase_(\d+)', epoch_path)
        elif "musicalpiece" in epoch_path:
            epoch_type = 'musicalpiece'
            match = re.search(r'musicalpiece_(\d+)', epoch_path)
        else:
            raise ValueError("Unknown epoch type in path: " + epoch_path)
        
        # Extract the number (x) from the match if it exists
        number = match.group(1) if match else "unknown"


        # Use the determined epoch type to create sub-epochs
        sub_epochs_data = create_sub_epochs(epochs, epoch_type=epoch_type)

        # Extract valence and arousal data for each sub-epoch   
        valance_arousal = []
        for sub_epoch in sub_epochs_data:
            valence_data, arousal_data = extract_valence_arousal_for_epoch(sub_epoch, raw)
            valance_arousal.append(np.hstack((valence_data.mean(), arousal_data.mean())))
        valance_arousal = np.array(valance_arousal)

        # Extract features from the sub-epochs
        features = extract_features(sub_epochs_data, sfreq, epoch_type)

        # Check if features were extracted
        if features is None or len(features) == 0:
            print(f"Subject {subject} Epoch {epoch_index+1} has no valid features. Skipping.")
            continue

        # Handle NaNs before scaling
        imputer = SimpleImputer(missing_values=np.nan, strategy='median')
        features_imputed = imputer.fit_transform(features)

        # Proceed with scaling and UMAP if features are present
        features_scaled = scaler.fit_transform(features_imputed)

        # Apply UMAP with the specific parameters for each epoch type
        umap = UMAP(**umap_params)
        embedding = umap.fit_transform(features_scaled)


        # Before computing Spearman correlation, check for constant arrays
        if np.std(umap_distances.ravel()) == 0 or np.std(valence_arousal_distances.ravel()) == 0:
            # Handle the case where there's no variation in distances
            # For example:
            correlation = np.nan
            p_value = np.nan
        else:
            correlation, p_value = spearmanr(umap_distances.ravel(), valence_arousal_distances.ravel())
        # Compute Spearman correlation between UMAP and valence/arousal distances
        umap_distances = squareform(pdist(embedding, 'euclidean'))
        valence_arousal_distances = squareform(pdist(valance_arousal, 'euclidean'))
        correlation, p_value = spearmanr(umap_distances.ravel(), valence_arousal_distances.ravel())
        print(f"Spearman correlation between UMAP distances and valence/arousal distances: {correlation} (p-value: {p_value})")
        
        # Print shapes of the arrays to debug
        print(f"Subject {subject} Epoch {epoch_index+1}")
        print("embedding.shape:", embedding.shape)
        print("valance_arousal.shape:", valance_arousal.shape)
        
        # Before normalization, check if the peak-to-peak value is zero and handle it
        if np.ptp(valance_arousal[:, 0]) == 0 or np.ptp(valance_arousal[:, 1]) == 0:
            # Handle the case where there's no variation in the valence or arousal
            # Maybe set normalized values to zero or skip this epoch
            # For example:
            valence_normalized = np.zeros_like(valance_arousal[:, 0])
            arousal_normalized = np.zeros_like(valance_arousal[:, 1])
        else:
            valence_normalized = (valance_arousal[:, 0] - np.min(valance_arousal[:, 0])) / np.ptp(valance_arousal[:, 0])
            arousal_normalized = (valance_arousal[:, 1] - np.min(valance_arousal[:, 1])) / np.ptp(valance_arousal[:, 1])    

        # Compute a color index for each point
        valence_colors = np.int_(valence_normalized * (num_colors - 1))
        arousal_colors = np.int_(arousal_normalized * (num_colors - 1))
        colors = valence_colors + arousal_colors * num_colors

        # Create a figure and a set of subplots
        fig, ax = plt.subplots(figsize=(12, 10))

        # Plot UMAP embedding
        scatter = ax.scatter(embedding[:, 0], embedding[:, 1], c=colors, cmap=cmap, s=5)

        # Add colorbar using the computed 2D color mapping
        norm = plt.Normalize(vmin=0, vmax=num_colors**2 - 1)
        sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
        sm.set_array([])
        cbar = fig.colorbar(sm, ax=ax, orientation='horizontal')
        cbar.set_label('Valence and Arousal')

        # Set the title and labels with the new naming convention
        ax.set_title(f'UMAP of Subject {subject} {epoch_type} {number} with Valence and Arousal Overlays')

        # Remove the axes tick marks and labels
        ax.set_xticks([])
        ax.set_yticks([])

        # Show plot
        plt.show()

In [ ]:
import mne
import numpy as np
from mne import Epochs, find_events
from sklearn.preprocessing import StandardScaler
from umap import UMAP
import matplotlib.pyplot as plt
from scipy.signal import welch
from scipy.stats import pearsonr
from sklearn.impute import SimpleImputer
from scipy.spatial.distance import pdist, squareform
from scipy.stats import spearmanr

# Define paths to the processed epochs and the raw EEG files
processed_epochs_paths = {
    '02': [
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\02\\02_musicalphrase_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\02\\02_musicalphrase_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\02\\02_musicalpiece_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\02\\02_musicalpiece_2-epo.fif',
    ],
    '04': [
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\04\\04_musicalphrase_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\04\\04_musicalphrase_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\04\\04_musicalphrase_3-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\04\\04_musicalpiece_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\04\\04_musicalpiece_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\04\\04_musicalpiece_3-epo.fif'
    ],
    '05': [
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\05\\05_musicalphrase_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\05\\05_musicalphrase_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\05\\05_musicalphrase_3-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\05\\05_musicalpiece_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\05\\05_musicalpiece_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\05\\05_musicalpiece_3-epo.fif'
    ],
    '10': [
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\10\\10_musicalphrase_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\10\\10_musicalphrase_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\10\\10_musicalphrase_3-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\10\\10_musicalpiece_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\10\\10_musicalpiece_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\10\\10_musicalpiece_3-epo.fif'
    ],
    '17': [
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\17\\17_musicalphrase_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\17\\17_musicalphrase_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\17\\17_musicalphrase_3-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\17\\17_musicalpiece_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\17\\17_musicalpiece_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\17\\17_musicalpiece_3-epo.fif'
    ],
    '19': [
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\19\\19_musicalphrase_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\19\\19_musicalphrase_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\19\\19_musicalphrase_3-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\19\\19_musicalpiece_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\19\\19_musicalpiece_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\19\\19_musicalpiece_3-epo.fif'
    ],
    '21': [
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\21\\21_musicalphrase_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\21\\21_musicalphrase_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\21\\21_musicalphrase_3-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\21\\21_musicalpiece_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\21\\21_musicalpiece_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\21\\21_musicalpiece_3-epo.fif'
    ],
}

raw_file_paths = {
    '02': 'C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-02\\eeg\\sub-02_task-classicalMusic_eeg.edf',
    '04': 'C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-04\\eeg\\sub-04_task-classicalMusic_eeg.edf',
    '05': 'C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-05\\eeg\\sub-05_task-classicalMusic_eeg.edf',
    '10': 'C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-10\\eeg\\sub-10_task-classicalMusic_eeg.edf',
    '17': 'C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-17\\eeg\\sub-17_task-classicalMusic_eeg.edf',
    '19': 'C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-19\\eeg\\sub-19_task-classicalMusic_eeg.edf',
    '21': 'C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-21\\eeg\\sub-21_task-classicalMusic_eeg.edf'
}

def create_sub_epochs(epochs, epoch_type):
    sub_epochs_data = []
    sfreq = epochs.info['sfreq']  # Sampling frequency
    
    if epoch_type == 'musicalphrase':
        sub_epoch_length = 0.500  # 1s for phrases
    elif epoch_type == 'musicalpiece':
        sub_epoch_length = 1 # 5s for pieces
    else:
        raise ValueError(f"Unknown epoch type: {epoch_type}")

    # Calculate the number of samples for the given sub_epoch_length
    sub_epoch_samples = int(sub_epoch_length * sfreq)

    # Create sub-epochs
    for start_time in np.arange(0, epochs.tmax, sub_epoch_length):
        start_idx = int(start_time * sfreq)
        end_idx = start_idx + sub_epoch_samples
        if end_idx > len(epochs.times):  # Ensure we don't go past the end
            break

        # Ensure indices are integers and correct slicing
        start_idx = int(start_idx)
        end_idx = int(end_idx)
        # Append sub-epoch data
        sub_epochs_data.append(epochs.copy().crop(tmin=epochs.times[start_idx], tmax=epochs.times[end_idx-1], include_tmax=True))
    
    return sub_epochs_data

def extract_valance_arousal_for_epoch(epoch, raw, valence_channel='ft_valance', arousal_channel='ft_arousal'):
    """
    Extracts valence and arousal data for a given epoch from the raw EDF file.
    
    :param epoch: The epoch for which to extract valence and arousal
    :param raw: The raw EDF data containing valence and arousal channels
    :param valence_channel: The name of the valence channel
    :param arousal_channel: The name of the arousal channel
    :return: A tuple containing the valence and arousal data arrays
    """
    # Get the start and end times of the epoch
    start, end = epoch.times[0], epoch.times[-1]
    start_sample, end_sample = int(start * raw.info['sfreq']), int(end * raw.info['sfreq'])
    
    # Extract valence and arousal data for the duration of the epoch
    valence_data = raw.copy().pick_channels([valence_channel]).get_data(start=start_sample, stop=end_sample)
    arousal_data = raw.copy().pick_channels([arousal_channel]).get_data(start=start_sample, stop=end_sample)
    
    return valence_data, arousal_data


def extract_features(sub_epochs_data, sfreq, epoch_type):
    features = []
    # Define the frequency bands
    bands = {'delta': (0.5, 4), 'theta': (4, 8), 'alpha': (8, 13)}
    spectral_features = []
    pcc_features = []

    # Calculate the number of samples for the given sub_epoch lengths based on the sampling frequency
    if epoch_type == 'musicalphrase':
        sub_epoch_length = 0.500  # 1s for phrases
        num_samples = int(sub_epoch_length * sfreq)
    elif epoch_type == 'musicalpiece':
        sub_epoch_length = 1  # 5s for pieces
        num_samples = int(sub_epoch_length * sfreq)
    else:
        raise ValueError(f"Unknown epoch type: {epoch_type}")

    for sub_epoch in sub_epochs_data:
        # Extract the data from the EpochsFIF object
        sub_epoch_data = sub_epoch.get_data()

        # Ensure nperseg does not exceed the number of samples in the sub-epoch
        nperseg = min(sub_epoch_data.shape[2], num_samples)  # The third dimension contains the time samples

        # Compute the PSD for the sub-epoch using Welch's method with the given nperseg
        f, Pxx = welch(sub_epoch_data.squeeze(), sfreq, nperseg=nperseg)

        # Calculate the spectral power in each band
        band_powers = []
        for band_name, (fmin, fmax) in bands.items():
            band_freq_indices = (f >= fmin) & (f <= fmax)
            if band_freq_indices.any():
                band_power = Pxx[:, band_freq_indices].mean(axis=-1)
                band_powers.append(band_power)
            else:
                band_powers.append(np.nan)  # Handle empty band

        band_powers = np.hstack(band_powers) if band_powers else np.array([np.nan, np.nan, np.nan])
        spectral_features.append(band_powers)

        # Calculate Pearson correlation coefficient between pairs of channels for the sub-epoch
        pcc = np.corrcoef(sub_epoch_data.squeeze(), rowvar=False)  # Use the numpy array here
        pcc = pcc[np.triu_indices_from(pcc, k=1)]  # Take the upper triangle of the matrix, excluding the diagonal
        pcc_features.extend(pcc)

    spectral_features = np.vstack(spectral_features)
    num_sub_epochs = len(sub_epochs_data)
    pcc_features = np.array(pcc_features).reshape(num_sub_epochs, -1)
    features = np.hstack([spectral_features, pcc_features]) if pcc_features.size else spectral_features

    # After computing features, handle NaN values
    imputer = SimpleImputer(missing_values=np.nan, strategy='median')
    features_imputed = imputer.fit_transform(features)

    return features_imputed

# Define the rescaling function
def rescale_to_range(data, new_min, new_max):
    old_min, old_max = np.min(data), np.max(data)
    return new_min + (data - old_min) * (new_max - new_min) / (old_max - old_min)


# Initialize the scaler
scaler = StandardScaler()

# Create a 2D colormap
num_colors = 256  # Number of colors to use in each dimension
cmap = plt.get_cmap('Spectral', num_colors**2)  # Use a colormap with enough colors

# Read raw data only once for each subject
raw_data = {}

for subject, raw_file_path in raw_file_paths.items():
    raw_data[subject] = mne.io.read_raw_edf(raw_file_path, preload=True)
    # Rescale channels only once
    for ch in ['ft_valance', 'ft_arousal']:
        if ch in raw_data[subject].ch_names:
            data = raw_data[subject].get_data(picks=ch)
            rescaled_data = rescale_to_range(data, -5, 5)
            raw_data[subject]._data[raw_data[subject].ch_names.index(ch), :] = rescaled_data

# Now process each epoch
for subject, epoch_paths in processed_epochs_paths.items():
    for epoch_index, epoch_path in enumerate(epoch_paths):
        epochs = mne.read_epochs(epoch_path, preload=True)
        raw = raw_data[subject]  # Use the preloaded and rescaled raw data

        sfreq = epochs.info['sfreq']  # Get the sampling frequency from the epochs

        # Determine the type of epoch based on the file name
        if "musicalphrase" in epoch_path:
            epoch_type = 'musicalphrase'
            umap_params = {'n_neighbors': 5, 'min_dist': 0.1, 'n_components': 2, 'random_state': 42}
        elif "musicalpiece" in epoch_path:
            epoch_type = 'musicalpiece'
            umap_params = {'n_neighbors': 5, 'min_dist': 0.1, 'n_components': 2, 'random_state': 42}
        else:
            raise ValueError("Unknown epoch type in path: " + epoch_path)

        # Use the determined epoch type to create sub-epochs
        sub_epochs_data = create_sub_epochs(epochs, epoch_type=epoch_type)

        # Extract valence and arousal data for each sub-epoch   
        valance_arousal = []
        for sub_epoch in sub_epochs_data:
            valence_data, arousal_data = extract_valence_arousal_for_epoch(sub_epoch, raw)
            valance_arousal.append(np.hstack((valence_data.mean(), arousal_data.mean())))
        valance_arousal = np.array(valance_arousal)

        # Extract features from the sub-epochs
        features = extract_features(sub_epochs_data, sfreq, epoch_type)

        # Check if features were extracted
        if features is None or len(features) == 0:
            print(f"Subject {subject} Epoch {epoch_index+1} has no valid features. Skipping.")
            continue

        # Handle NaNs before scaling
        imputer = SimpleImputer(missing_values=np.nan, strategy='median')
        features_imputed = imputer.fit_transform(features)

        # Proceed with scaling and UMAP if features are present
        features_scaled = scaler.fit_transform(features_imputed)

        # Apply UMAP with the specific parameters for each epoch type
        umap = UMAP(**umap_params)
        embedding = umap.fit_transform(features_scaled)

        # Compute Spearman correlation between UMAP and valence/arousal distances
        umap_distances = squareform(pdist(embedding, 'euclidean'))
        valence_arousal_distances = squareform(pdist(valance_arousal, 'euclidean'))
        correlation, p_value = spearmanr(umap_distances.ravel(), valence_arousal_distances.ravel())
        print(f"Spearman correlation between UMAP distances and valence/arousal distances: {correlation} (p-value: {p_value})")
        
        # Print shapes of the arrays to debug
        print(f"Subject {subject} Epoch {epoch_index+1}")
        print("embedding.shape:", embedding.shape)
        print("valance_arousal.shape:", valance_arousal.shape)
        
        # Normalize valence and arousal
        valence_normalized = (valance_arousal[:, 0] - np.min(valance_arousal[:, 0])) / np.ptp(valance_arousal[:, 0])
        arousal_normalized = (valance_arousal[:, 1] - np.min(valance_arousal[:, 1])) / np.ptp(valance_arousal[:, 1])

        # Compute a color index for each point
        valence_colors = np.int_(valence_normalized * (num_colors - 1))
        arousal_colors = np.int_(arousal_normalized * (num_colors - 1))
        colors = valence_colors + arousal_colors * num_colors

        # Create a figure and a set of subplots
        fig, ax = plt.subplots(figsize=(12, 10))

        # Plot UMAP embedding
        scatter = ax.scatter(embedding[:, 0], embedding[:, 1], c=colors, cmap=cmap, s=5)

        # Add colorbar using the computed 2D color mapping
        norm = plt.Normalize(vmin=0, vmax=num_colors**2 - 1)
        sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
        sm.set_array([])
        cbar = fig.colorbar(sm, ax=ax, orientation='horizontal')
        cbar.set_label('Valence and Arousal')

        ax.set_title(f'UMAP of Subject {subject} Epoch {epoch_index+1} with Valence and Arousal Overlays')
        ax.set_xlabel('UMAP Component 1')
        ax.set_ylabel('UMAP Component 2')

        # Show plot
        plt.show()

In [ ]:
import mne
import numpy as np
from mne import Epochs, find_events
from sklearn.preprocessing import StandardScaler
from umap import UMAP
import matplotlib.pyplot as plt
from scipy.signal import welch
from scipy.stats import pearsonr
from sklearn.impute import SimpleImputer



# Define paths to the processed epochs and the raw EEG files
processed_epochs_paths = {
    '02': [
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\02\\02_musicalphrase_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\02\\02_musicalphrase_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\02\\02_musicalpiece_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\02\\02_musicalpiece_2-epo.fif',
    ],
    '04': [
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\04\\04_musicalphrase_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\04\\04_musicalphrase_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\04\\04_musicalphrase_3-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\04\\04_musicalpiece_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\04\\04_musicalpiece_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\04\\04_musicalpiece_3-epo.fif'
    ],
    '05': [
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\05\\05_musicalphrase_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\05\\05_musicalphrase_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\05\\05_musicalphrase_3-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\05\\05_musicalpiece_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\05\\05_musicalpiece_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\05\\05_musicalpiece_3-epo.fif'
    ],
    '10': [
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\10\\10_musicalphrase_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\10\\10_musicalphrase_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\10\\10_musicalphrase_3-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\10\\10_musicalpiece_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\10\\10_musicalpiece_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\10\\10_musicalpiece_3-epo.fif'
    ],
    '17': [
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\17\\17_musicalphrase_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\17\\17_musicalphrase_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\17\\17_musicalphrase_3-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\17\\17_musicalpiece_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\17\\17_musicalpiece_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\17\\17_musicalpiece_3-epo.fif'
    ],
    '19': [
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\19\\19_musicalphrase_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\19\\19_musicalphrase_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\19\\19_musicalphrase_3-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\19\\19_musicalpiece_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\19\\19_musicalpiece_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\19\\19_musicalpiece_3-epo.fif'
    ],
    '21': [
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\21\\21_musicalphrase_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\21\\21_musicalphrase_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\21\\21_musicalphrase_3-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\21\\21_musicalpiece_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\21\\21_musicalpiece_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\21\\21_musicalpiece_3-epo.fif'
    ],
}

def create_sub_epochs(epochs, epoch_type):
    sub_epochs_data = []
    sfreq = epochs.info['sfreq']  # Sampling frequency
    
    if epoch_type == 'musicalphrase':
        sub_epoch_length = 0.1  # 100 ms for phrases
    elif epoch_type == 'musicalpiece':
        sub_epoch_length = 0.5  # 1 second for pieces
    else:
        raise ValueError(f"Unknown epoch type: {epoch_type}")

    # Calculate the number of samples for the given sub_epoch_length
    sub_epoch_samples = int(sub_epoch_length * sfreq)

    # Create sub-epochs
    for start_time in np.arange(0, epochs.tmax, sub_epoch_length):
        start_idx = int(start_time * sfreq)
        end_idx = start_idx + sub_epoch_samples
        if end_idx > len(epochs.times):  # Ensure we don't go past the end
            break

        # Ensure indices are integers and correct slicing
        start_idx = int(start_idx)
        end_idx = int(end_idx)
        # Append sub-epoch data
        sub_epochs_data.append(epochs.copy().crop(tmin=epochs.times[start_idx], tmax=epochs.times[end_idx-1], include_tmax=True))
    
    return sub_epochs_data


def extract_features(sub_epochs_data, sfreq, epoch_type):
    features = []
    # Define the frequency bands
    bands = {'delta': (0.5, 4), 'theta': (4, 8), 'alpha': (8, 13)}
    spectral_features = []
    pcc_features = []

    # Calculate the number of samples for the given sub_epoch lengths based on the sampling frequency
    if epoch_type == 'musicalphrase':
        sub_epoch_length = 0.1  # 100 ms for phrases
        num_samples = int(sub_epoch_length * sfreq)
    elif epoch_type == 'musicalpiece':
        sub_epoch_length = 1.0  # 1 second for pieces
        num_samples = int(sub_epoch_length * sfreq)
    else:
        raise ValueError(f"Unknown epoch type: {epoch_type}")

    for sub_epoch in sub_epochs_data:
        # Extract the data from the EpochsFIF object
        sub_epoch_data = sub_epoch.get_data()

        # Ensure nperseg does not exceed the number of samples in the sub-epoch
        nperseg = min(sub_epoch_data.shape[2], num_samples)  # The third dimension contains the time samples

        # Compute the PSD for the sub-epoch using Welch's method with the given nperseg
        f, Pxx = welch(sub_epoch_data.squeeze(), sfreq, nperseg=nperseg)

        # Calculate the spectral power in each band
        band_powers = []
        for band_name, (fmin, fmax) in bands.items():
            band_freq_indices = (f >= fmin) & (f <= fmax)
            if band_freq_indices.any():
                band_power = Pxx[:, band_freq_indices].mean(axis=-1)
                band_powers.append(band_power)
            else:
                band_powers.append(np.nan)  # Handle empty band

        band_powers = np.hstack(band_powers) if band_powers else np.array([np.nan, np.nan, np.nan])
        spectral_features.append(band_powers)

        # Calculate Pearson correlation coefficient between pairs of channels for the sub-epoch
        pcc = np.corrcoef(sub_epoch_data.squeeze(), rowvar=False)  # Use the numpy array here
        pcc = pcc[np.triu_indices_from(pcc, k=1)]  # Take the upper triangle of the matrix, excluding the diagonal
        pcc_features.extend(pcc)

    spectral_features = np.vstack(spectral_features)
    num_sub_epochs = len(sub_epochs_data)
    pcc_features = np.array(pcc_features).reshape(num_sub_epochs, -1)
    features = np.hstack([spectral_features, pcc_features]) if pcc_features.size else spectral_features

    # After computing features, handle NaN values
    imputer = SimpleImputer(missing_values=np.nan, strategy='median')
    features_imputed = imputer.fit_transform(features)

    return features_imputed


# Initialize the scaler
scaler = StandardScaler()

# Process each subject's epochs
for subject, epoch_paths in processed_epochs_paths.items():
    for epoch_index, epoch_path in enumerate(epoch_paths):
        epochs = mne.read_epochs(epoch_path, preload=True)
        sfreq = epochs.info['sfreq']  # Get the sampling frequency from the epochs

        # Determine the type of epoch based on the file name
        if "musicalphrase" in epoch_path:
            epoch_type = 'musicalphrase'
        elif "musicalpiece" in epoch_path:
            epoch_type = 'musicalpiece'
        else:
            raise ValueError("Unknown epoch type in path: " + epoch_path)

        # Use the determined epoch type to create sub-epochs
        sub_epochs_data = create_sub_epochs(epochs, epoch_type=epoch_type)

        # Extract features from the sub-epochs
        features = extract_features(sub_epochs_data, sfreq, epoch_type)

        # Check if features were extracted
        if features is None or len(features) == 0:
            print(f"Subject {subject} Epoch {epoch_index+1} has no valid features. Skipping.")
            continue

        # Handle NaNs before scaling
        imputer = SimpleImputer(missing_values=np.nan, strategy='median')
        features_imputed = imputer.fit_transform(features)

        # Proceed with scaling and UMAP if features are present
        features_scaled = scaler.fit_transform(features_imputed)
        umap = UMAP(n_neighbors=5, min_dist=0.5, spread=1.0, metric='manhattan')
        embedding = umap.fit_transform(features_scaled)
        
        plt.figure(figsize=(12, 10))
        plt.scatter(embedding[:, 0], embedding[:, 1], s=5)
        plt.title(f"UMAP of Subject {subject} Epoch {epoch_index+1}")
        plt.xlabel('UMAP Component 1')
        plt.ylabel('UMAP Component 2')
        plt.show()

In [ ]:
import mne
import numpy as np
from mne import Epochs, find_events
from sklearn.preprocessing import StandardScaler
from umap import UMAP
import matplotlib.pyplot as plt
from scipy.signal import welch
from scipy.stats import pearsonr
from sklearn.impute import SimpleImputer



# Define paths to the processed epochs and the raw EEG files
processed_epochs_paths = {
    '02': [
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\02\\02_musicalphrase_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\02\\02_musicalphrase_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\02\\02_musicalpiece_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\02\\02_musicalpiece_2-epo.fif',
    ],
    '04': [
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\04\\04_musicalphrase_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\04\\04_musicalphrase_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\04\\04_musicalphrase_3-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\04\\04_musicalpiece_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\04\\04_musicalpiece_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\04\\04_musicalpiece_3-epo.fif'
    ],
    '05': [
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\05\\05_musicalphrase_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\05\\05_musicalphrase_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\05\\05_musicalphrase_3-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\05\\05_musicalpiece_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\05\\05_musicalpiece_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\05\\05_musicalpiece_3-epo.fif'
    ],
    '10': [
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\10\\10_musicalphrase_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\10\\10_musicalphrase_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\10\\10_musicalphrase_3-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\10\\10_musicalpiece_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\10\\10_musicalpiece_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\10\\10_musicalpiece_3-epo.fif'
    ],
    '17': [
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\17\\17_musicalphrase_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\17\\17_musicalphrase_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\17\\17_musicalphrase_3-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\17\\17_musicalpiece_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\17\\17_musicalpiece_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\17\\17_musicalpiece_3-epo.fif'
    ],
    '19': [
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\19\\19_musicalphrase_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\19\\19_musicalphrase_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\19\\19_musicalphrase_3-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\19\\19_musicalpiece_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\19\\19_musicalpiece_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\19\\19_musicalpiece_3-epo.fif'
    ],
    '21': [
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\21\\21_musicalphrase_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\21\\21_musicalphrase_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\21\\21_musicalphrase_3-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\21\\21_musicalpiece_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\21\\21_musicalpiece_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\21\\21_musicalpiece_3-epo.fif'
    ],
}

def create_sub_epochs(epochs, epoch_type):
    sub_epochs_data = []
    sfreq = epochs.info['sfreq']  # Sampling frequency
    
    if epoch_type == 'musicalphrase':
        sub_epoch_length = 0.1  # 100 ms for phrases
    elif epoch_type == 'musicalpiece':
        sub_epoch_length = 0.5  # 1 second for pieces
    else:
        raise ValueError(f"Unknown epoch type: {epoch_type}")

    # Calculate the number of samples for the given sub_epoch_length
    sub_epoch_samples = int(sub_epoch_length * sfreq)

    # Create sub-epochs
    for start_time in np.arange(0, epochs.tmax, sub_epoch_length):
        start_idx = int(start_time * sfreq)
        end_idx = start_idx + sub_epoch_samples
        if end_idx > len(epochs.times):  # Ensure we don't go past the end
            break

        # Ensure indices are integers and correct slicing
        start_idx = int(start_idx)
        end_idx = int(end_idx)
        # Append sub-epoch data
        sub_epochs_data.append(epochs.copy().crop(tmin=epochs.times[start_idx], tmax=epochs.times[end_idx-1], include_tmax=True))
    
    return sub_epochs_data


def extract_features(sub_epochs_data, sfreq, epoch_type):
    features = []
    # Define the frequency bands
    bands = {'delta': (0.5, 4), 'theta': (4, 8), 'alpha': (8, 13)}
    spectral_features = []
    pcc_features = []

    # Calculate the number of samples for the given sub_epoch lengths based on the sampling frequency
    if epoch_type == 'musicalphrase':
        sub_epoch_length = 0.1  # 100 ms for phrases
        num_samples = int(sub_epoch_length * sfreq)
    elif epoch_type == 'musicalpiece':
        sub_epoch_length = 0.5  # 1 second for pieces
        num_samples = int(sub_epoch_length * sfreq)
    else:
        raise ValueError(f"Unknown epoch type: {epoch_type}")

    for sub_epoch in sub_epochs_data:
        # Extract the data from the EpochsFIF object
        sub_epoch_data = sub_epoch.get_data()

        # Ensure nperseg does not exceed the number of samples in the sub-epoch
        nperseg = min(sub_epoch_data.shape[2], num_samples)  # The third dimension contains the time samples

        # Compute the PSD for the sub-epoch using Welch's method with the given nperseg
        f, Pxx = welch(sub_epoch_data.squeeze(), sfreq, nperseg=nperseg)

        # Calculate the spectral power in each band
        band_powers = []
        for band_name, (fmin, fmax) in bands.items():
            band_freq_indices = (f >= fmin) & (f <= fmax)
            if band_freq_indices.any():
                band_power = Pxx[:, band_freq_indices].mean(axis=-1)
                band_powers.append(band_power)
            else:
                band_powers.append(np.nan)  # Handle empty band

        band_powers = np.hstack(band_powers) if band_powers else np.array([np.nan, np.nan, np.nan])
        spectral_features.append(band_powers)

        # Calculate Pearson correlation coefficient between pairs of channels for the sub-epoch
        pcc = np.corrcoef(sub_epoch_data.squeeze(), rowvar=False)  # Use the numpy array here
        pcc = pcc[np.triu_indices_from(pcc, k=1)]  # Take the upper triangle of the matrix, excluding the diagonal
        pcc_features.extend(pcc)

    spectral_features = np.vstack(spectral_features)
    num_sub_epochs = len(sub_epochs_data)
    pcc_features = np.array(pcc_features).reshape(num_sub_epochs, -1)
    features = np.hstack([spectral_features, pcc_features]) if pcc_features.size else spectral_features

    # After computing features, handle NaN values
    imputer = SimpleImputer(missing_values=np.nan, strategy='median')
    features_imputed = imputer.fit_transform(features)

    return features_imputed


# Initialize the scaler
scaler = StandardScaler()

# Process each subject's epochs
for subject, epoch_paths in processed_epochs_paths.items():
    for epoch_index, epoch_path in enumerate(epoch_paths):
        epochs = mne.read_epochs(epoch_path, preload=True)
        sfreq = epochs.info['sfreq']  # Get the sampling frequency from the epochs

        # Determine the type of epoch based on the file name
        if "musicalphrase" in epoch_path:
            epoch_type = 'musicalphrase'
        elif "musicalpiece" in epoch_path:
            epoch_type = 'musicalpiece'
        else:
            raise ValueError("Unknown epoch type in path: " + epoch_path)

        # Use the determined epoch type to create sub-epochs
        sub_epochs_data = create_sub_epochs(epochs, epoch_type=epoch_type)

        # Extract features from the sub-epochs
        features = extract_features(sub_epochs_data, sfreq, epoch_type)

        # Check if features were extracted
        if features is None or len(features) == 0:
            print(f"Subject {subject} Epoch {epoch_index+1} has no valid features. Skipping.")
            continue

        # Handle NaNs before scaling
        imputer = SimpleImputer(missing_values=np.nan, strategy='median')
        features_imputed = imputer.fit_transform(features)

        # Proceed with scaling and UMAP if features are present
        features_scaled = scaler.fit_transform(features_imputed)
        umap = UMAP(n_neighbors=5, min_dist=0.1, n_components=2, random_state=42)
        embedding = umap.fit_transform(features_scaled)
        
        plt.figure(figsize=(12, 10))
        plt.scatter(embedding[:, 0], embedding[:, 1], s=5)
        plt.title(f"UMAP of Subject {subject} Epoch {epoch_index+1}")
        plt.xlabel('UMAP Component 1')
        plt.ylabel('UMAP Component 2')
        plt.show()

In [ ]:
import mne
import numpy as np
from mne import Epochs, find_events
from sklearn.preprocessing import StandardScaler
from umap import UMAP
import matplotlib.pyplot as plt
from scipy.signal import welch
from scipy.stats import pearsonr
from sklearn.impute import SimpleImputer



# Define paths to the processed epochs and the raw EEG files
processed_epochs_paths = {
    '02': [
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\02\\02_musicalphrase_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\02\\02_musicalphrase_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\02\\02_musicalpiece_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\02\\02_musicalpiece_2-epo.fif',
    ],
    '04': [
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\04\\04_musicalphrase_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\04\\04_musicalphrase_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\04\\04_musicalphrase_3-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\04\\04_musicalpiece_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\04\\04_musicalpiece_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\04\\04_musicalpiece_3-epo.fif'
    ],
    '05': [
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\05\\05_musicalphrase_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\05\\05_musicalphrase_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\05\\05_musicalphrase_3-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\05\\05_musicalpiece_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\05\\05_musicalpiece_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\05\\05_musicalpiece_3-epo.fif'
    ],
    '10': [
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\10\\10_musicalphrase_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\10\\10_musicalphrase_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\10\\10_musicalphrase_3-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\10\\10_musicalpiece_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\10\\10_musicalpiece_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\10\\10_musicalpiece_3-epo.fif'
    ],
    '17': [
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\17\\17_musicalphrase_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\17\\17_musicalphrase_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\17\\17_musicalphrase_3-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\17\\17_musicalpiece_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\17\\17_musicalpiece_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\17\\17_musicalpiece_3-epo.fif'
    ],
    '19': [
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\19\\19_musicalphrase_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\19\\19_musicalphrase_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\19\\19_musicalphrase_3-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\19\\19_musicalpiece_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\19\\19_musicalpiece_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\19\\19_musicalpiece_3-epo.fif'
    ],
    '21': [
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\21\\21_musicalphrase_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\21\\21_musicalphrase_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\21\\21_musicalphrase_3-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\21\\21_musicalpiece_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\21\\21_musicalpiece_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalpieces_data\\21\\21_musicalpiece_3-epo.fif'
    ],
}

def create_sub_epochs(epochs, epoch_type):
    sub_epochs_data = []
    sfreq = epochs.info['sfreq']  # Sampling frequency
    
    if epoch_type == 'musicalphrase':
        sub_epoch_length = 0.1  # 100 ms for phrases
    elif epoch_type == 'musicalpiece':
        sub_epoch_length = 0.5  # 1 second for pieces
    else:
        raise ValueError(f"Unknown epoch type: {epoch_type}")

    # Calculate the number of samples for the given sub_epoch_length
    sub_epoch_samples = int(sub_epoch_length * sfreq)

    # Create sub-epochs
    for start_time in np.arange(0, epochs.tmax, sub_epoch_length):
        start_idx = int(start_time * sfreq)
        end_idx = start_idx + sub_epoch_samples
        if end_idx > len(epochs.times):  # Ensure we don't go past the end
            break

        # Ensure indices are integers and correct slicing
        start_idx = int(start_idx)
        end_idx = int(end_idx)
        # Append sub-epoch data
        sub_epochs_data.append(epochs.copy().crop(tmin=epochs.times[start_idx], tmax=epochs.times[end_idx-1], include_tmax=True))
    
    return sub_epochs_data


def extract_features(sub_epochs_data, sfreq, epoch_type):
    features = []
    # Define the frequency bands
    bands = {'delta': (0.5, 4), 'theta': (4, 8), 'alpha': (8, 13)}
    spectral_features = []
    pcc_features = []

    # Calculate the number of samples for the given sub_epoch lengths based on the sampling frequency
    if epoch_type == 'musicalphrase':
        sub_epoch_length = 0.1  # 100 ms for phrases
        num_samples = int(sub_epoch_length * sfreq)
    elif epoch_type == 'musicalpiece':
        sub_epoch_length = 1.0  # 1 second for pieces
        num_samples = int(sub_epoch_length * sfreq)
    else:
        raise ValueError(f"Unknown epoch type: {epoch_type}")

    for sub_epoch in sub_epochs_data:
        # Extract the data from the EpochsFIF object
        sub_epoch_data = sub_epoch.get_data()

        # Ensure nperseg does not exceed the number of samples in the sub-epoch
        nperseg = min(sub_epoch_data.shape[2], num_samples)  # The third dimension contains the time samples

        # Compute the PSD for the sub-epoch using Welch's method with the given nperseg
        f, Pxx = welch(sub_epoch_data.squeeze(), sfreq, nperseg=nperseg)

        # Calculate the spectral power in each band
        band_powers = []
        for band_name, (fmin, fmax) in bands.items():
            band_freq_indices = (f >= fmin) & (f <= fmax)
            if band_freq_indices.any():
                band_power = Pxx[:, band_freq_indices].mean(axis=-1)
                band_powers.append(band_power)
            else:
                band_powers.append(np.nan)  # Handle empty band

        band_powers = np.hstack(band_powers) if band_powers else np.array([np.nan, np.nan, np.nan])
        spectral_features.append(band_powers)

        # Calculate Pearson correlation coefficient between pairs of channels for the sub-epoch
        pcc = np.corrcoef(sub_epoch_data.squeeze(), rowvar=False)  # Use the numpy array here
        pcc = pcc[np.triu_indices_from(pcc, k=1)]  # Take the upper triangle of the matrix, excluding the diagonal
        pcc_features.extend(pcc)

    spectral_features = np.vstack(spectral_features)
    num_sub_epochs = len(sub_epochs_data)
    pcc_features = np.array(pcc_features).reshape(num_sub_epochs, -1)
    features = np.hstack([spectral_features, pcc_features]) if pcc_features.size else spectral_features

    # After computing features, handle NaN values
    imputer = SimpleImputer(missing_values=np.nan, strategy='median')
    features_imputed = imputer.fit_transform(features)

    return features_imputed


# Initialize the scaler
scaler = StandardScaler()

# Process each subject's epochs
for subject, epoch_paths in processed_epochs_paths.items():
    for epoch_index, epoch_path in enumerate(epoch_paths):
        epochs = mne.read_epochs(epoch_path, preload=True)
        sfreq = epochs.info['sfreq']  # Get the sampling frequency from the epochs

        # Determine the type of epoch based on the file name
        if "musicalphrase" in epoch_path:
            epoch_type = 'musicalphrase'
        elif "musicalpiece" in epoch_path:
            epoch_type = 'musicalpiece'
        else:
            raise ValueError("Unknown epoch type in path: " + epoch_path)

        # Use the determined epoch type to create sub-epochs
        sub_epochs_data = create_sub_epochs(epochs, epoch_type=epoch_type)

        # Extract features from the sub-epochs
        features = extract_features(sub_epochs_data, sfreq, epoch_type)

        # Check if features were extracted
        if features is None or len(features) == 0:
            print(f"Subject {subject} Epoch {epoch_index+1} has no valid features. Skipping.")
            continue

        # Handle NaNs before scaling
        imputer = SimpleImputer(missing_values=np.nan, strategy='median')
        features_imputed = imputer.fit_transform(features)

        # Proceed with scaling and UMAP if features are present
        features_scaled = scaler.fit_transform(features_imputed)
        umap = UMAP(n_neighbors=5, min_dist=0.1, n_components=2, random_state=42)
        embedding = umap.fit_transform(features_scaled)
        
        plt.figure(figsize=(12, 10))
        plt.scatter(embedding[:, 0], embedding[:, 1], s=5)
        plt.title(f"UMAP of Subject {subject} Epoch {epoch_index+1}")
        plt.xlabel('UMAP Component 1')
        plt.ylabel('UMAP Component 2')
        plt.show()


In [ ]:
import mne
import numpy as np
from sklearn.preprocessing import StandardScaler
from scipy.stats import pearsonr
from umap import UMAP
import matplotlib.pyplot as plt

# Define paths to the processed epochs and the raw EEG files
processed_epochs_paths = {
    '02': [
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\02\\02_musicalphrase_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\02\\02_musicalphrase_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalmoments_data\\02\\02_musicalmoment_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalmoments_data\\02\\02_musicalmoment_2-epo.fif',
    ],
    '04': [
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\04\\04_musicalphrase_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\04\\04_musicalphrase_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\04\\04_musicalphrase_3-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalmoments_data\\04\\04_musicalmoment_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalmoments_data\\04\\04_musicalmoment_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalmoments_data\\04\\04_musicalmoment_3-epo.fif'
    ],
    '05': [
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\05\\05_musicalphrase_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\05\\05_musicalphrase_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\05\\05_musicalphrase_3-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalmoments_data\\05\\05_musicalmoment_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalmoments_data\\05\\05_musicalmoment_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalmoments_data\\05\\05_musicalmoment_3-epo.fif'
    ],
    '10': [
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\10\\10_musicalphrase_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\10\\10_musicalphrase_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\10\\10_musicalphrase_3-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalmoments_data\\10\\10_musicalmoment_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalmoments_data\\10\\10_musicalmoment_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalmoments_data\\10\\10_musicalmoment_3-epo.fif'
    ],
    '17': [
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\17\\17_musicalphrase_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\17\\17_musicalphrase_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\17\\17_musicalphrase_3-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalmoments_data\\17\\17_musicalmoment_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalmoments_data\\17\\17_musicalmoment_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalmoments_data\\17\\17_musicalmoment_3-epo.fif'
    ],
    '19': [
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\19\\19_musicalphrase_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\19\\19_musicalphrase_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\19\\19_musicalphrase_3-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalmoments_data\\19\\19_musicalmoment_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalmoments_data\\19\\19_musicalmoment_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalmoments_data\\19\\19_musicalmoment_3-epo.fif'
    ],
    '21': [
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\21\\21_musicalphrase_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\21\\21_musicalphrase_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\21\\21_musicalphrase_3-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalmoments_data\\21\\21_musicalmoment_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalmoments_data\\21\\21_musicalmoment_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalmoments_data\\21\\21_musicalmoment_3-epo.fif'
    ],
}

# Function to extract features from epochs
def extract_features(epochs):
    features = []
    
    # Extract spectral power using the new compute_psd method
    psd = epochs.compute_psd(fmin=0.5, fmax=47, method='welch')
    psds, freqs = psd.get_data(return_freqs=True)
    print("PSDs shape:", psds.shape)  # Debugging line
    print("Frequencies:", freqs)  # Debugging line

    # Define the frequency bands
    bands = {'delta': (0.5, 4), 'theta': (4, 8), 'alpha': (8, 13)}

    # Calculate the spectral power in each band
    spectral_features = []
    for band_name, (fmin, fmax) in bands.items():
        psd_band = psds[:, :, (freqs >= fmin) & (freqs <= fmax)].mean(axis=-1)
        print(f"Band {band_name} feature shape:", psd_band.shape)  # Debugging line
        spectral_features.append(psd_band)
    
    spectral_features = np.hstack(spectral_features)
    print("Spectral features shape:", spectral_features.shape)  # Debugging line

    # Calculate Pearson correlation coefficient between pairs of channels for each epoch
    pcc_features = []
    for epoch_data in epochs.get_data():
        pcc, _ = pearsonr(epoch_data[0], epoch_data[1])
        pcc_features.append(pcc)
    
    pcc_features = np.array(pcc_features).reshape(-1, 1)
    print("PCC features shape:", pcc_features.shape)  # Debugging line
    print("PCC features:", pcc_features)  # Debugging line

    # Combine spectral and PCC features
    features = np.hstack([spectral_features, pcc_features])
    
    return features


# Initialize the scaler
scaler = StandardScaler()

# Feature array placeholder
all_features = []

# Loop over each subject and each epoch path
for subject, epoch_paths in processed_epochs_paths.items():
    for epoch_index, epoch_path in enumerate(epoch_paths):
        # Load the epochs
        epochs = mne.read_epochs(epoch_path, preload=True)
        
        # Extract features
        features = extract_features(epochs)
        
        # Scale the features for the current epoch        
        features_scaled = scaler.fit_transform(features)

        # Initialize UMAP with desired parameters for the current epoch
        umap = UMAP(n_neighbors=5, min_dist=0.5, n_components=2, random_state=42)
        
        # Fit UMAP to the scaled features for the current epoch
        embedding = umap.fit_transform(features_scaled)
        
        # Now you can create a UMAP visualization for the current epoch
        plt.figure(figsize=(12, 10))
        plt.scatter(embedding[:, 0], embedding[:, 1], s=5)  # s is the size of points
        plt.title(f"UMAP of Subject {subject} Epoch {epoch_index+1}")
        plt.xlabel('UMAP Component 1')
        plt.ylabel('UMAP Component 2')
        plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


# Define paths to the processed epochs and the raw EEG files
processed_epochs_paths = {
    '02': [
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\02\\02_musicalphrase_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\02\\02_musicalphrase_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalmoments_data\\02\\02_musicalmoment_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalmoments_data\\02\\02_musicalmoment_2-epo.fif',
    ],
    '04': [
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\04\\04_musicalphrase_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\04\\04_musicalphrase_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\04\\04_musicalphrase_3-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalmoments_data\\04\\04_musicalmoment_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalmoments_data\\04\\04_musicalmoment_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalmoments_data\\04\\04_musicalmoment_3-epo.fif'
    ],
    '05': [
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\05\\05_musicalphrase_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\05\\05_musicalphrase_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\05\\05_musicalphrase_3-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalmoments_data\\05\\05_musicalmoment_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalmoments_data\\05\\05_musicalmoment_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalmoments_data\\05\\05_musicalmoment_3-epo.fif'
    ],
    '10': [
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\10\\10_musicalphrase_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\10\\10_musicalphrase_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\10\\10_musicalphrase_3-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalmoments_data\\10\\10_musicalmoment_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalmoments_data\\10\\10_musicalmoment_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalmoments_data\\10\\10_musicalmoment_3-epo.fif'
    ],
    '17': [
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\17\\17_musicalphrase_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\17\\17_musicalphrase_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\17\\17_musicalphrase_3-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalmoments_data\\17\\17_musicalmoment_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalmoments_data\\17\\17_musicalmoment_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalmoments_data\\17\\17_musicalmoment_3-epo.fif'
    ],
    '19': [
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\19\\19_musicalphrase_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\19\\19_musicalphrase_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\19\\19_musicalphrase_3-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalmoments_data\\19\\19_musicalmoment_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalmoments_data\\19\\19_musicalmoment_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalmoments_data\\19\\19_musicalmoment_3-epo.fif'
    ],
    '21': [
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\21\\21_musicalphrase_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\21\\21_musicalphrase_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalphrases_data\\21\\21_musicalphrase_3-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalmoments_data\\21\\21_musicalmoment_1-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalmoments_data\\21\\21_musicalmoment_2-epo.fif',
        'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\musicalmoments_data\\21\\21_musicalmoment_3-epo.fif'
    ],
}
# Use the existing `extract_features` function from your code.

# This function will help us check the variance and visualize the features
def check_variance_and_plot(features):
    variances = np.var(features, axis=0)  # Calculate variance along the column
    print("Variances of features:", variances)

    # Plot the variance
    plt.figure(figsize=(10, 5))
    plt.bar(range(len(variances)), variances)
    plt.title('Feature Variances')
    plt.xlabel('Feature index')
    plt.ylabel('Variance')
    plt.show()

    # Plot histograms for each feature
    num_features = features.shape[1]
    fig, axes = plt.subplots(num_features, 1, figsize=(10, num_features*2))
    for i in range(num_features):
        axes[i].hist(features[:, i], bins=50)
        axes[i].set_title(f'Feature {i} distribution')
    plt.tight_layout()
    plt.show()

# Placeholder for all features for debugging
all_features_debug = {}

# Iterate over each subject and epoch file
for subject, epoch_paths in processed_epochs_paths.items():
    for epoch_path in epoch_paths:
        # Load the epochs
        epochs = mne.read_epochs(epoch_path, preload=True)

        # Extract features
        features = extract_features(epochs)

        # Check variance and plot for the current epoch
        check_variance_and_plot(features)

        # Prepare the path string for the dictionary key
        # Replace backslashes with forward slashes to avoid the SyntaxError in f-strings
        path_key = epoch_path.replace('\\', '/')
        
        # Store the features for further debugging if needed
        all_features_debug[f"{subject}_{path_key.split('/')[-1]}"] = features

# Now you can inspect the variances and distributions to see if the features are too similar

In [ ]:
raw file:
 'C:\Users\josep\Music&Topology\all-subjects\sub-02\eeg\sub-02_task-classicalMusic_eeg.edf'
corresponding timestamps for epochs:
17.092-175.484 (s) for 'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data\\sub02\\sub02_music-and-reporting_p3-epo.fif'
175.484-308.868 (s) for 'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data\sub02\\sub02_music-and-reporting_p6-or-p2-epo.fif'

raw file:
'C:\Users\josep\Music&Topology\all-subjects\sub-04\eeg\sub-04_task-classicalMusic_eeg.edf'
corresponding timestamps for epochs:
175.555-337.511 (s) for 'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data\\sub04\\sub04_music-and-reporting_p5-undecided-epo.fif'
337.511-472.401 (s) for 'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data\\sub04\\sub04_music-and-reporting_p6-or-p2-epo.fif'
605.756-761.398 (s) for 'C:\\Users\\josep\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data\\sub04\\sub04_music-and-reporting_p3-epo.fif'
1059.81-1225.402 (s) for 'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data\\sub04\\sub04_music-and-reporting_p5-undecided-epo.fif'

raw file:
'C:\Users\josep\Music&Topology\all-subjects\sub-05\eeg\sub-05_task-classicalMusic_eeg.edf'
corresponding timestamps for epochs:
14.594-148.165 (s) for 'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data\\sub05\\sub05_music-and-reporting_p6-or-p2-epo.fif'
282.772-448.714 (s) for 'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data\\sub05\\sub05_music-and-reporting_p5-undecided-epo.fif'
908.811-1065.856 (s) for 'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data\\sub05\\sub05_music-and-reporting_p3-epo.fif'
1224.351-1389.993 (s) for 'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data\\sub05\\sub05_music-and-reporting_p5-undecided-epo.fif'

raw file:
'C:\Users\josep\Music&Topology\all-subjects\sub-10\eeg\sub-10_task-classicalMusic_eeg.edf'
corresponding timestamps for epochs:
14.906-149.429 (s) for 'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data\\sub10\\sub10_music-and-reporting_p6-or-p2-epo.fif'
447.09-602.681 (s) for 'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data\\sub10\\sub10_music-and-reporting_p3-epo.fif'
769.324-932.846 (s) for C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data\\sub10\\sub10_music-and-reporting_p5-undecided-epo.fif'
1089.656-1254.697 (s) for C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data\\sub10\\sub10_music-and-reporting_p5-undecided-epo.fif'

raw file:
'C:\Users\josep\Music&Topology\all-subjects\sub-17\eeg\sub-17_task-classicalMusic_eeg.edf'
corresponding timestamps for epochs:
14.109-172.254 (s) for 'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data\\sub17\\sub17_music-and-reporting_p3-epo.fif'
172.254-339.316 (s) for 'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data\\sub17\\sub17_music-and-reporting_p5-undecided-epo.fif'
628.262-791.817 (s) for 'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data\\sub17\\sub17_music-and-reporting_p5-undecided-epo.fif'
791.817-925.556 (s) for 'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data\\sub17\\sub17_music-and-reporting_p6-or-p2-epo.fif'

raw file:
'C:\Users\josep\Music&Topology\all-subjects\sub-19\eeg\sub-19_task-classicalMusic_eeg.edf'
corresponding timestamps for epochs:
179.071-344.363 (s) for 'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data\\sub19\\sub19_music-and-reporting_p5-undecided-epo.fif'
502.826-637.032 (s) for 'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data\\sub19\\sub19_music-and-reporting_p6-or-p2-epo.fif'
637.032-794.395 (s) for  'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data\\sub19\\sub19_music-and-reporting_p3-epo.fif'
1124.542-1288.438 (s) for 'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data\\sub19\\sub19_music-and-reporting_p5-undecided-epo.fif'

raw file:
'C:\Users\josep\Music&Topology\all-subjects\sub-21\eeg\sub-21_task-classicalMusic_eeg.edf'
corresponding timestamps for epochs:
180.259-345.484 (s) for 'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data\\sub21\\sub21_music-and-reporting_p5-undecided-epo.fif'
502.01-636.067 (s) for 'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data\\sub21\\sub21_music-and-reporting_p6-or-p2-epo.fif''
636.067-791.376 (s) for 'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data\\sub21\\sub21_music-and-reporting_p3-epo.fif'
1121.608-1285.497 (s) for 'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data\\sub21\\sub21_music-and-reporting_p5-undecided-epo.fif'

In [ ]:
# List of music-listening trialsegs files to visualize
epoch_files = [
]

# step 4: trial segmentations layout

In [ ]:
'''
general outline:
    1) Load the raw EEG data and events from the provided annotations or events.tsv file.
    2) Plot the non-EEG channels (like 'music', 'trialtype', 'ft_valence', 'ft_arousal') to visually inspect them.
    3) Based on your visual inspection, decide on the events that correspond to each trial's start and end.
    4) Epoch the data based on these manually determined events.
    5) Label each epoch based on the conditions you described earlier.
    6) Save the labeled epochs.

selected_subjects = ['02', '04', '05', '10', '16', '17', '19', '21']
'''

# step 4 —> getting sample numbers 

When running the DL model (first time), have 'C:\\Users\\josep\\Music&Topology\\all work ;)\Analysis1\processed-for-DL\sub-xx_task-classicalMusic_eeg_raw-with-ICA-weights-filtered-and-referenced_DL.fif' for edf_file_path
; After running the DL model and having obtained its classifications for the music-listening data, change to 'C:\\Users\\josep\\Music&Topology\\all work ;)\Analysis1\\processed-for-analysis\\sub-xx_task-classicalMusic_eeg_raw-with-ICA-weights-filtered-and-referenced_analysis.fif'

In [ ]:
'''
music_listening_piece1_epochs = all_epochs_sub02['label == "music-listening_piece1"']
music_and_reporting_piece2_epochs = all_epochs_sub02['label == "music-and-reporting_piece2"']
reporting_only_epochs = all_epochs_sub02['label == "reporting-only_piece3"']
'''

# subject 2's sample numbers

edf_file_path = 'C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-02\\eeg\\sub-02_task-classicalMusic_eeg.edf'  
raw = mne.io.read_raw_edf(edf_file_path, preload=True)

sfreq = raw.info['sfreq']  # get the sampling frequency

start_trial_1_sub02 = int(17.092 * sfreq)
end_trial_1_sub02 = int(175.484 * sfreq)

start_trial_2_sub02 = int(175.484 * sfreq)
end_trial_2_sub02 = int(308.868 * sfreq)

start_trial_3_sub02 = int(308.868 * sfreq)
end_trial_3_sub02 = int(474.824 * sfreq)

start_trial_4_sub02 = int(474.824 * sfreq)
end_trial_4_sub02 = int(639.524 * sfreq)

start_trial_5_sub02 = int(639.524 * sfreq)
end_trial_5_sub02 = int(803.843 * sfreq)

start_trial_6_sub02 = int(803.843 * sfreq)
end_trial_6_sub02 = int(971.251 * sfreq)

start_trial_7_sub02 = int(971.251 * sfreq)
end_trial_7_sub02 = int(1129.759 * sfreq)

start_trial_8_sub02 = int(1129.759 * sfreq)
end_trial_8_sub02 = int(1294.88 * sfreq)

start_trial_9_sub02 = int(1294.88 * sfreq)
end_trial_9_sub02 = int(1451.318 * sfreq)

start_trial_10_sub02 = int(1451.318 * sfreq)
end_trial_10_sub02 = int(1585.837 * sfreq)

start_trial_11_sub02 = int(1585.837 * sfreq)
end_trial_11_sub02 = int(1750.842 * sfreq)

start_trial_12_sub02 = int(1750.842 * sfreq)
end_trial_12_sub02 = int(raw.times[-1] * sfreq)

In [ ]:
# previous value for start_trial_11_sub02: 1585837
raw.times[-1]

In [ ]:
# subject 4's sample numbers

edf_file_path = 'C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-04\\eeg\\sub-04_task-classicalMusic_eeg.edf'  
raw = mne.io.read_raw_edf(edf_file_path, preload=True)

sfreq = raw.info['sfreq']  # get the sampling frequency

start_trial_1_sub04 = int(17.397 * sfreq)
end_trial_1_sub04 = int(173.555 * sfreq)

start_trial_2_sub04 = int(173.555 * sfreq)
end_trial_2_sub04 = int(337.511 * sfreq)

start_trial_3_sub04 = int(337.511 * sfreq)
end_trial_3_sub04 = int(472.401 * sfreq)

start_trial_4_sub04 = int(472.401 * sfreq)
end_trial_4_sub04 = int(605.756 * sfreq)

start_trial_5_sub04 = int(605.756 * sfreq)
end_trial_5_sub04 = int(761.398 * sfreq)

start_trial_6_sub04 = int(761.398 * sfreq)
end_trial_6_sub04 = int(895.254 * sfreq)

start_trial_7_sub04 = int(895.254 * sfreq)
end_trial_7_sub04 = int(1059.81 * sfreq)

start_trial_8_sub04 = int(1059.81 * sfreq)
end_trial_8_sub04 = int(1225.402 * sfreq)

start_trial_9_sub04 = int(1225.402 * sfreq)
end_trial_9_sub04 = int(1392.295 * sfreq)

start_trial_10_sub04 = int(1392.295 * sfreq)
end_trial_10_sub04 = int(1550.308 * sfreq)

start_trial_11_sub04 = int(1550.308 * sfreq)
end_trial_11_sub04 = int(1713.795 * sfreq)

start_trial_12_sub04 = int(1713.795 * sfreq)
end_trial_12_sub04 = int(raw.times[-1] * sfreq)

In [ ]:
# subject 5's sample numbers

edf_file_path = 'C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-05\\eeg\\sub-05_task-classicalMusic_eeg.edf'  
raw = mne.io.read_raw_edf(edf_file_path, preload=True)

sfreq = raw.info['sfreq']  # get the sampling frequency

start_trial_1_sub05 = int(14.594 * sfreq)
end_trial_1_sub05 = int(148.165 * sfreq)

start_trial_2_sub05 = int(148.165 * sfreq)
end_trial_2_sub05 = int(282.772 * sfreq)

start_trial_3_sub05 = int(282.772 * sfreq)
end_trial_3_sub05 = int(448.714 * sfreq)

start_trial_4_sub05 = int(448.714 * sfreq)
end_trial_4_sub05 = int(611.869 * sfreq)

start_trial_5_sub05 = int(611.869 * sfreq)
end_trial_5_sub05 = int(775.824 * sfreq)

start_trial_6_sub05 = int(775.824 * sfreq)
end_trial_6_sub05 = int(908.811 * sfreq)

start_trial_7_sub05 = int(908.811 * sfreq)
end_trial_7_sub05 = int(1065.856 * sfreq)

start_trial_8_sub05 = int(1065.856 * sfreq)
end_trial_8_sub05 = int(1224.351 * sfreq)

start_trial_9_sub05 = int(1224.351 * sfreq)
end_trial_9_sub05 = int(1389.993 * sfreq)

start_trial_10_sub05 = int(1389.993 * sfreq)
end_trial_10_sub05 = int(1555.836 * sfreq)

start_trial_11_sub05 = int(1555.836 * sfreq)
end_trial_11_sub05 = int(1722.094 * sfreq)

start_trial_12_sub05 = int(1722.094 * sfreq)
end_trial_12_sub05 = int(raw.times[-1] * sfreq)

In [ ]:
# subject 10's sample numbers

edf_file_path = 'C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-10\\eeg\\sub-10_task-classicalMusic_eeg.edf'  
raw = mne.io.read_raw_edf(edf_file_path, preload=True)

start_trial_1_sub10 = int(14.906 * sfreq)
end_trial_1_sub10 = int(149.429 * sfreq)

start_trial_2_sub10 = int(149.429 * sfreq)
end_trial_2_sub10 = int(282.784 * sfreq)

start_trial_3_sub10 = int(282.784 * sfreq)
end_trial_3_sub10 = int(447.09 * sfreq)

start_trial_4_sub10 = int(447.09 * sfreq)
end_trial_4_sub10 = int(602.681 * sfreq)

start_trial_5_sub10 = int(602.681 * sfreq)
end_trial_5_sub10 = int(769.324 * sfreq)

start_trial_6_sub10 = int(769.324 * sfreq)
end_trial_6_sub10 = int(932.846 * sfreq)

start_trial_7_sub10 = int(932.846 * sfreq)
end_trial_7_sub10 = int(1089.656 * sfreq)

start_trial_8_sub10 = int(1089.656 * sfreq)
end_trial_8_sub10 = int(1254.697 * sfreq)

start_trial_9_sub10 = int(1254.697 * sfreq)
end_trial_9_sub10 = int(1410.689 * sfreq)

start_trial_10_sub10 = int(1410.689 * sfreq)
end_trial_10_sub10 = int(1543.478 * sfreq)

start_trial_11_sub10 = int(1543.478 * sfreq)
end_trial_11_sub10 = int(1709.218 * sfreq)

start_trial_12_sub10 = int(1709.218 * sfreq)
end_trial_12_sub10 = int(raw.times[-1] * sfreq)

In [ ]:
# subject 16's sample numbers

edf_file_path = 'C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-16\\eeg\\sub-16_task-classicalMusic_eeg.edf'  
raw = mne.io.read_raw_edf(edf_file_path, preload=True)

start_trial_1_sub16 = int(14.197 * sfreq)
end_trial_1_sub16 = int(147.534 * sfreq)

start_trial_2_sub16 = int(147.534 * sfreq)
end_trial_2_sub16 = int(282.259 * sfreq)

start_trial_3_sub16 = int(282.259 * sfreq)
end_trial_3_sub16 = int(447.867 * sfreq)

start_trial_4_sub16 = int(447.867 * sfreq)
end_trial_4_sub16 = int(603.475 * sfreq)

start_trial_5_sub16 = int(603.475 * sfreq)
end_trial_5_sub16 = int(759.684 * sfreq)

start_trial_6_sub16 = int(759.684 * sfreq)
end_trial_6_sub16 = int(924.14 * sfreq)

start_trial_7_sub16 = int(924.14 * sfreq)
end_trial_7_sub16 = int(1091.552 * sfreq)

start_trial_8_sub16 = int(1091.552 * sfreq)
end_trial_8_sub16 = int(1256.392 * sfreq)

start_trial_9_sub16 = int(1256.392 * sfreq)
end_trial_9_sub16 = int(1413.454 * sfreq)

start_trial_10_sub16 = int(1413.454 * sfreq)
end_trial_10_sub16 = int(1578.294 * sfreq)

start_trial_11_sub16 = int(1578.294 * sfreq)
end_trial_11_sub16 = int(1711.449 * sfreq)

start_trial_12_sub16 = int(1711.449 * sfreq)
end_trial_12_sub16 = int(raw.times[-1] * sfreq)

In [ ]:
# subject 17's sample numbers

edf_file_path = 'C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-17\\eeg\\sub-17_task-classicalMusic_eeg.edf'  
raw = mne.io.read_raw_edf(edf_file_path, preload=True)

start_trial_1_sub17 = int(14.109 * sfreq)
end_trial_1_sub17 = int(172.254 * sfreq)

start_trial_2_sub17 = int(172.254 * sfreq)
end_trial_2_sub17 = int(339.316 * sfreq)

start_trial_3_sub17 = int(339.316 * sfreq)
end_trial_3_sub17 = int(471.986 * sfreq)

start_trial_4_sub17 = int(471.986 * sfreq)
end_trial_4_sub17 = int(628.262 * sfreq)

start_trial_5_sub17 = int(628.262 * sfreq)
end_trial_5_sub17 = int(791.817 * sfreq)

start_trial_6_sub17 = int(791.817 * sfreq)
end_trial_6_sub17 = int(925.556 * sfreq)

start_trial_7_sub17 = int(925.556 * sfreq)
end_trial_7_sub17 = int(1091.365 * sfreq)

start_trial_8_sub17 = int(1091.365 * sfreq)
end_trial_8_sub17 = int(1247.358 * sfreq)

start_trial_9_sub17 = int(1247.358 * sfreq)
end_trial_9_sub17 = int(1412.299 * sfreq)

start_trial_10_sub17 = int(1412.299 * sfreq)
end_trial_10_sub17 = int(1577.756 * sfreq)

start_trial_11_sub17 = int(1577.756 * sfreq)
end_trial_11_sub17 = int(1711.364 * sfreq)

start_trial_12_sub17 = int(1711.364 * sfreq)
end_trial_12_sub17 = int(raw.times[-1] * sfreq)

In [ ]:
# subject 19's sample numbers

edf_file_path = 'C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-19\\eeg\\sub-19_task-classicalMusic_eeg.edf'  
raw = mne.io.read_raw_edf(edf_file_path, preload=True)

start_trial_1_sub19 = int(12.362 * sfreq)
end_trial_1_sub19 = int(179.071 * sfreq)

start_trial_2_sub19 = int(179.071 * sfreq)
end_trial_2_sub19 = int(344.363 * sfreq)

start_trial_3_sub19 = int(344.363 * sfreq)
end_trial_3_sub19 = int(502.826 * sfreq)

start_trial_4_sub19 = int(502.826 * sfreq)
end_trial_4_sub19 = int(637.032 * sfreq)

start_trial_5_sub19 = int(637.032 * sfreq)
end_trial_5_sub19 = int(794.395 * sfreq)

start_trial_6_sub19 = int(794.395 * sfreq)
end_trial_6_sub19 = int(959.235 * sfreq)

start_trial_7_sub19 = int(959.235 * sfreq)
end_trial_7_sub19 = int(1124.542 * sfreq)

start_trial_8_sub19 = int(1124.542 * sfreq)
end_trial_8_sub19 = int(1288.348 * sfreq)

start_trial_9_sub19 = int(1288.348 * sfreq)
end_trial_9_sub19 = int(1422.805 * sfreq)

start_trial_10_sub19 = int(1422.805 * sfreq)
end_trial_10_sub19 = int(1557.011 * sfreq)

start_trial_11_sub19 = int(1557.011 * sfreq)
end_trial_11_sub19 = int(1713.354 * sfreq)

start_trial_12_sub19 = int(1713.354 * sfreq)
end_trial_12_sub19 = int(raw.times[-1] * sfreq)

In [ ]:
# subject 21's sample numbers

edf_file_path = 'C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-21\\eeg\\sub-21_task-classicalMusic_eeg.edf'  
raw = mne.io.read_raw_edf(edf_file_path, preload=True)

start_trial_1_sub21 = int(13.466 * sfreq)
end_trial_1_sub21 = int(180.259 * sfreq)

start_trial_2_sub21 = int(180.259 * sfreq)
end_trial_2_sub21 = int(345.484 * sfreq)

start_trial_3_sub21 = int(345.484 * sfreq)
end_trial_3_sub21 = int(502.01 * sfreq)

start_trial_4_sub21 = int(502.01 * sfreq)
end_trial_4_sub21 = int(636.067 * sfreq)

start_trial_5_sub21 = int(636.067 * sfreq)
end_trial_5_sub21 = int(791.376 * sfreq)

start_trial_6_sub21 = int(791.376 * sfreq)
end_trial_6_sub21 = int(956.333 * sfreq)

start_trial_7_sub21 = int(956.333 * sfreq)
end_trial_7_sub21 = int(1121.608 * sfreq)

start_trial_8_sub21 = int(1121.608 * sfreq)
end_trial_8_sub21 = int(1285.497 * sfreq)

start_trial_9_sub21 = int(1285.497 * sfreq)
end_trial_9_sub21 = int(1417.766 * sfreq)

start_trial_10_sub21 = int(1417.766 * sfreq)
end_trial_10_sub21 = int(1550.101 * sfreq)

start_trial_11_sub21 = int(1550.101 * sfreq)
end_trial_11_sub21 = int(1708.518 * sfreq)

start_trial_12_sub21 = int(1708.518 * sfreq)
end_trial_12_sub21 = int(raw.times[-1] * sfreq)
'''
11/4 update: it's fine for DL, I'm not using the whole trial anyways; 
for steps 1-2, proceed as is as I can look at where the music cuts off, if it does, when I visualize trial-by-trial;
for step 3, just look at where the music cuts off and use that estimated timepoint to get the new piece name, no need for 亂七八糟與以下的

----

Also, to clarify a concern I had from the above, when we said the following:

'To add the end_trial_12_sub02 event based on the final timepoint of the music channel, you can access the last timepoint of a channel by looking at the raw data's times attribute. Since you're interested in the final timepoint of the music channel, you'd find the last entry in the raw.times array and then multiply by the sampling frequency to get the sample number.
Here's the adjusted code for subject 02, taking into account the extraction of end_trial_12_sub02:
# extracting sample numbers

sfreq = raw.info['sfreq'] 
...
start_trial_11_sub02 = int(1585.837 * sfreq)
end_trial_11_sub02 = int(1750.842 * sfreq)

start_trial_12_sub02 = int(1750.842 * sfreq)
end_trial_12_sub02 = int(raw.times[-1] * sfreq)  # Last timepoint of the music channel converted to sample number

# labeling trialsegs

events_sub02 = [
    # ... [previous events here]
    [start_trial_12_sub02, 0, 1],
    [end_trial_12_sub02, 0, 2]
]

# Rest of the code remains unchanged

I am unsure that the code we had will actually get the last timepoint of the music channel I meant, the non-EEG music data channel, within the raw '...classical_music_eeg.edf' file. We previously had code that might be re-adaptable to get the last timepoint of the non-EEG channel of each subject; perhaps just make non_eeg_channels = ['music'] and get the last timepoint right under? 

for dir in selected_subject_dirs:
    # Get only the classical music EDF files
    classical_music_files = list(dir.glob('eeg/*task-classicalMusic_eeg.edf'))
    
    for edf_file in classical_music_files:
        # Load the data
        raw = mne.io.read_raw_edf(edf_file, preload=True)
        
        # Get the indices of the non-EEG channels
        non_eeg_channels = ['ft_valance', 'ft_arousal', 'music', 'trialtype']

Then perhaps, I can just select from a list of these last timepoints for each subject's _end_trial_12; example from subject 21 to change:

# edit this...

...
start_trial_12_sub21 = int(1708.518 * sfreq)
end_trial_12_sub21 = int(raw.times[-1] * sfreq)
'''

# step 4 —> subject-by-subject cells for trialsegs

In [ ]:
''' 
Subject 2!
'''

import os
import matplotlib.pyplot as plt
import mne
import numpy as np

# Load your data
edf_file_path = 'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\raw-with-ICA-weights\\raw-with-ICA-weights\\sub-02_task-classicalMusic_eeg_raw-with-ICA-weights.fif' 
raw = mne.io.read_raw_fif(edf_file_path, preload=True)

# Define events manually based on step 2
# Here's a hypothetical example:
events_sub02 = [
    [start_trial_1_sub02, 0, 1],
    [end_trial_1_sub02, 0, 2],
    [start_trial_2_sub02, 0, 1],
    [end_trial_2_sub02, 0, 2],
    [start_trial_3_sub02, 0, 1],
    [end_trial_3_sub02, 0, 2],
    [start_trial_4_sub02, 0, 1],
    [end_trial_4_sub02, 0, 2],
    [start_trial_5_sub02, 0, 1],
    [end_trial_5_sub02, 0, 2],
    [start_trial_6_sub02, 0, 1],
    [end_trial_6_sub02, 0, 2],
    [start_trial_7_sub02, 0, 1],
    [end_trial_7_sub02, 0, 2],
    [start_trial_8_sub02, 0, 1],
    [end_trial_8_sub02, 0, 2],
    [start_trial_9_sub02, 0, 1],
    [end_trial_9_sub02, 0, 2],
    [start_trial_10_sub02, 0, 1],
    [end_trial_10_sub02, 0, 2],
    [start_trial_11_sub02, 0, 1],
    [end_trial_11_sub02, 0, 2],
    [start_trial_12_sub02, 0, 1],
    [end_trial_12_sub02, 0, 2]
]

# Create trial segmentations (trialsegs) based on these events
trial_starts_sub02 = [event[0] for event in events_sub02 if event[2] == 1]
trial_ends_sub02 = [event[0] for event in events_sub02 if event[2] == 2]

tmax_values_sub02 = [(end - start) / raw.info['sfreq'] for start, end in zip(trial_starts_sub02, trial_ends_sub02)]

trialsegs_sub02 = []
for i, start in enumerate(trial_starts_sub02):
    epochs_temp_sub02 = mne.Epochs(raw, events=[events_sub02[i*2]], event_id={'start': 1}, tmin=0, tmax=tmax_values_sub02[i], baseline=None, preload=True)
    trialsegs_sub02.append(epochs_temp_sub02)

# Manually label each trialseg
labels_sub02 = []
for ts in trialsegs_sub02:
    label = input(f"Enter label for trialseg {len(labels_sub02)+1}: ")  # This will prompt you to enter a label for each trialseg
    labels_sub02.append(label)

# Assign metadata to each individual mne.Epochs object
for i, ts in enumerate(trialsegs_sub02):
    ts.metadata = pd.DataFrame({'label': [labels_sub02[i]]})

# Create a folder for the subject's trialsegs if it doesn't exist
subject_id = "sub02"
trialsegs_output_folder = os.path.join("C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data", subject_id)
if not os.path.exists(trialsegs_output_folder):
    os.makedirs(trialsegs_output_folder)

# Save each labeled trialseg using its label as the name
for ts, label in zip(trialsegs_sub02, labels_sub02):
    safe_label = label.replace(" ", "_")  # to ensure filename compatibility
    ts.save(os.path.join(trialsegs_output_folder, f"{subject_id}_{safe_label}-epo.fif"), overwrite=True)


In [ ]:
''' 
Subject 4!
reporting-only

'''
# Load your data
edf_file_path = 'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\raw-with-ICA-weights\\raw-with-ICA-weights\\sub-04_task-classicalMusic_eeg_raw-with-ICA-weights.fif' 
raw = mne.io.read_raw_fif(edf_file_path, preload=True)

# Define events manually based on step 2
# Here's a hypothetical example:
events_sub04 = [
    [start_trial_1_sub04, 0, 1],
    [end_trial_1_sub04, 0, 2],
    [start_trial_2_sub04, 0, 1],
    [end_trial_2_sub04, 0, 2],
    [start_trial_3_sub04, 0, 1],
    [end_trial_3_sub04, 0, 2],
    [start_trial_4_sub04, 0, 1],
    [end_trial_4_sub04, 0, 2],
    [start_trial_5_sub04, 0, 1],
    [end_trial_5_sub04, 0, 2],
    [start_trial_6_sub04, 0, 1],
    [end_trial_6_sub04, 0, 2],
    [start_trial_7_sub04, 0, 1],
    [end_trial_7_sub04, 0, 2],
    [start_trial_8_sub04, 0, 1],
    [end_trial_8_sub04, 0, 2],
    [start_trial_9_sub04, 0, 1],
    [end_trial_9_sub04, 0, 2],
    [start_trial_10_sub04, 0, 1],
    [end_trial_10_sub04, 0, 2],
    [start_trial_11_sub04, 0, 1],
    [end_trial_11_sub04, 0, 2],
    [start_trial_12_sub04, 0, 1],
    [end_trial_12_sub04, 0, 2]
]

# Create trial segmentations (trialsegs) based on these events
trial_starts_sub04 = [event[0] for event in events_sub04 if event[2] == 1]
trial_ends_sub04 = [event[0] for event in events_sub04 if event[2] == 2]

tmax_values_sub04 = [(end - start) / raw.info['sfreq'] for start, end in zip(trial_starts_sub04, trial_ends_sub04)]

trialsegs_sub04 = []
for i, start in enumerate(trial_starts_sub04):
    epochs_temp_sub04 = mne.Epochs(raw, events=[events_sub04[i*2]], event_id={'start': 1}, tmin=0, tmax=tmax_values_sub04[i], baseline=None, preload=True)
    trialsegs_sub04.append(epochs_temp_sub04)

# Manually label each trialseg
labels_sub04 = []
for ts in trialsegs_sub04:
    label = input(f"Enter label for trialseg {len(labels_sub04)+1}: ")  # This will prompt you to enter a label for each trialseg
    labels_sub04.append(label)

# Assign metadata to each individual mne.Epochs object
for i, ts in enumerate(trialsegs_sub04):
    ts.metadata = pd.DataFrame({'label': [labels_sub04[i]]})

# Create a folder for the subject's trialsegs if it doesn't exist
subject_id = "sub04"
trialsegs_output_folder = os.path.join("C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data", subject_id)
if not os.path.exists(trialsegs_output_folder):
    os.makedirs(trialsegs_output_folder)

# Save each labeled trialseg using its label as the name
for ts, label in zip(trialsegs_sub04, labels_sub04):
    safe_label = label.replace(" ", "_")  # to ensure filename compatibility
    ts.save(os.path.join(trialsegs_output_folder, f"{subject_id}_{safe_label}-epo.fif"), overwrite=True)


In [ ]:
''' 
Subject 5!
'''

# Load your data
edf_file_path = 'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\raw-with-ICA-weights\\raw-with-ICA-weights\\sub-05_task-classicalMusic_eeg_raw-with-ICA-weights.fif' 
raw = mne.io.read_raw_fif(edf_file_path, preload=True)

# Define events manually based on step 2
# Here's a hypothetical example:
events_sub05 = [
    [start_trial_1_sub05, 0, 1],
    [end_trial_1_sub05, 0, 2],
    [start_trial_2_sub05, 0, 1],
    [end_trial_2_sub05, 0, 2],
    [start_trial_3_sub05, 0, 1],
    [end_trial_3_sub05, 0, 2],
    [start_trial_4_sub05, 0, 1],
    [end_trial_4_sub05, 0, 2],
    [start_trial_5_sub05, 0, 1],
    [end_trial_5_sub05, 0, 2],
    [start_trial_6_sub05, 0, 1],
    [end_trial_6_sub05, 0, 2],
    [start_trial_7_sub05, 0, 1],
    [end_trial_7_sub05, 0, 2],
    [start_trial_8_sub05, 0, 1],
    [end_trial_8_sub05, 0, 2],
    [start_trial_9_sub05, 0, 1],
    [end_trial_9_sub05, 0, 2],
    [start_trial_10_sub05, 0, 1],
    [end_trial_10_sub05, 0, 2],
    [start_trial_11_sub05, 0, 1],
    [end_trial_11_sub05, 0, 2],
    [start_trial_12_sub05, 0, 1],
    [end_trial_12_sub05, 0, 2]
]


# Create trial segmentations (trialsegs) based on these events
trial_starts_sub05 = [event[0] for event in events_sub05 if event[2] == 1]
trial_ends_sub05 = [event[0] for event in events_sub05 if event[2] == 2]

tmax_values_sub05 = [(end - start) / raw.info['sfreq'] for start, end in zip(trial_starts_sub05, trial_ends_sub05)]

trialsegs_sub05 = []
for i, start in enumerate(trial_starts_sub05):
    epochs_temp_sub05 = mne.Epochs(raw, events=[events_sub05[i*2]], event_id={'start': 1}, tmin=0, tmax=tmax_values_sub05[i], baseline=None, preload=True)
    trialsegs_sub05.append(epochs_temp_sub05)

# Manually label each trialseg
labels_sub05 = []
for ts in trialsegs_sub05:
    label = input(f"Enter label for trialseg {len(labels_sub05)+1}: ")  # This will prompt you to enter a label for each trialseg
    labels_sub05.append(label)

# Assign metadata to each individual mne.Epochs object
for i, ts in enumerate(trialsegs_sub05):
    ts.metadata = pd.DataFrame({'label': [labels_sub05[i]]})

# Create a folder for the subject's trialsegs if it doesn't exist
subject_id = "sub05"
trialsegs_output_folder = os.path.join("C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data", subject_id)
if not os.path.exists(trialsegs_output_folder):
    os.makedirs(trialsegs_output_folder)

# Save each labeled trialseg using its label as the name
for ts, label in zip(trialsegs_sub05, labels_sub05):
    safe_label = label.replace(" ", "_")  # to ensure filename compatibility
    ts.save(os.path.join(trialsegs_output_folder, f"{subject_id}_{safe_label}-epo.fif"), overwrite=True)

In [ ]:
''' 
Subject 10!
'''

# Load your data
edf_file_path = 'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\raw-with-ICA-weights\\raw-with-ICA-weights\\sub-10_task-classicalMusic_eeg_raw-with-ICA-weights.fif' 
raw = mne.io.read_raw_fif(edf_file_path, preload=True)

# Define events manually based on step 2
events_sub10 = [
    [start_trial_1_sub10, 0, 1],
    [end_trial_1_sub10, 0, 2],
    [start_trial_2_sub10, 0, 1],
    [end_trial_2_sub10, 0, 2],
    [start_trial_3_sub10, 0, 1],
    [end_trial_3_sub10, 0, 2],
    [start_trial_4_sub10, 0, 1],
    [end_trial_4_sub10, 0, 2],
    [start_trial_5_sub10, 0, 1],
    [end_trial_5_sub10, 0, 2],
    [start_trial_6_sub10, 0, 1],
    [end_trial_6_sub10, 0, 2],
    [start_trial_7_sub10, 0, 1],
    [end_trial_7_sub10, 0, 2],
    [start_trial_8_sub10, 0, 1],
    [end_trial_8_sub10, 0, 2],
    [start_trial_9_sub10, 0, 1],
    [end_trial_9_sub10, 0, 2],
    [start_trial_10_sub10, 0, 1],
    [end_trial_10_sub10, 0, 2],
    [start_trial_11_sub10, 0, 1],
    [end_trial_11_sub10, 0, 2],
    [start_trial_12_sub10, 0, 1],
    [end_trial_12_sub10, 0, 2]
]

# Create trial segmentations (trialsegs) based on these events
trial_starts_sub10 = [event[0] for event in events_sub10 if event[2] == 1]
trial_ends_sub10 = [event[0] for event in events_sub10 if event[2] == 2]

tmax_values_sub10 = [(end - start) / raw.info['sfreq'] for start, end in zip(trial_starts_sub10, trial_ends_sub10)]

trialsegs_sub10 = []
for i, start in enumerate(trial_starts_sub10):
    epochs_temp_sub10 = mne.Epochs(raw, events=[events_sub10[i*2]], event_id={'start': 1}, tmin=0, tmax=tmax_values_sub10[i], baseline=None, preload=True)
    trialsegs_sub10.append(epochs_temp_sub10)

# Manually label each trialseg
labels_sub10 = []
for ts in trialsegs_sub10:
    label = input(f"Enter label for trialseg {len(labels_sub10)+1}: ")  # This will prompt you to enter a label for each trialseg
    labels_sub10.append(label)

# Assign metadata to each individual mne.Epochs object
for i, ts in enumerate(trialsegs_sub10):
    ts.metadata = pd.DataFrame({'label': [labels_sub10[i]]})

# Create a folder for the subject's trialsegs if it doesn't exist
subject_id = "sub10"
trialsegs_output_folder = os.path.join("C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data", subject_id)
if not os.path.exists(trialsegs_output_folder):
    os.makedirs(trialsegs_output_folder)

# Save each labeled trialseg using its label as the name
for ts, label in zip(trialsegs_sub10, labels_sub10):
    safe_label = label.replace(" ", "_")  # to ensure filename compatibility
    ts.save(os.path.join(trialsegs_output_folder, f"{subject_id}_{safe_label}-epo.fif"), overwrite=True)


In [ ]:
''' 
Subject 17!
'''

# Load your data
edf_file_path = 'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\raw-with-ICA-weights\\raw-with-ICA-weights\\sub-17_task-classicalMusic_eeg_raw-with-ICA-weights.fif' 
raw = mne.io.read_raw_fif(edf_file_path, preload=True)

# Define events manually based on step 2
# Here's a hypothetical example:
events_sub17 = [
    [start_trial_1_sub17, 0, 1],
    [end_trial_1_sub17, 0, 2],
    [start_trial_2_sub17, 0, 1],
    [end_trial_2_sub17, 0, 2],
    [start_trial_3_sub17, 0, 1],
    [end_trial_3_sub17, 0, 2],
    [start_trial_4_sub17, 0, 1],
    [end_trial_4_sub17, 0, 2],
    [start_trial_5_sub17, 0, 1],
    [end_trial_5_sub17, 0, 2],
    [start_trial_6_sub17, 0, 1],
    [end_trial_6_sub17, 0, 2],
    [start_trial_7_sub17, 0, 1],
    [end_trial_7_sub17, 0, 2],
    [start_trial_8_sub17, 0, 1],
    [end_trial_8_sub17, 0, 2],
    [start_trial_9_sub17, 0, 1],
    [end_trial_9_sub17, 0, 2],
    [start_trial_10_sub17, 0, 1],
    [end_trial_10_sub17, 0, 2],
    [start_trial_11_sub17, 0, 1],
    [end_trial_11_sub17, 0, 2],
    [start_trial_12_sub17, 0, 1],
    [end_trial_12_sub17, 0, 2]
]

# Create trial segmentations (trialsegs) based on these events
trial_starts_sub17 = [event[0] for event in events_sub17 if event[2] == 1]
trial_ends_sub17 = [event[0] for event in events_sub17 if event[2] == 2]

tmax_values_sub17 = [(end - start) / raw.info['sfreq'] for start, end in zip(trial_starts_sub17, trial_ends_sub17)]

trialsegs_sub17 = []
for i, start in enumerate(trial_starts_sub17):
    epochs_temp_sub17 = mne.Epochs(raw, events=[events_sub17[i*2]], event_id={'start': 1}, tmin=0, tmax=tmax_values_sub17[i], baseline=None, preload=True)
    trialsegs_sub17.append(epochs_temp_sub17)

# Manually label each trialseg
labels_sub17 = []
for ts in trialsegs_sub17:
    label = input(f"Enter label for trialseg {len(labels_sub17)+1}: ")  # This will prompt you to enter a label for each trialseg
    labels_sub17.append(label)

# Assign metadata to each individual mne.Epochs object
for i, ts in enumerate(trialsegs_sub17):
    ts.metadata = pd.DataFrame({'label': [labels_sub17[i]]})

# Create a folder for the subject's trialsegs if it doesn't exist
subject_id = "sub17"
trialsegs_output_folder = os.path.join("C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data", subject_id)
if not os.path.exists(trialsegs_output_folder):
    os.makedirs(trialsegs_output_folder)

# Save each labeled trialseg using its label as the name
for ts, label in zip(trialsegs_sub17, labels_sub17):
    safe_label = label.replace(" ", "_")  # to ensure filename compatibility
    ts.save(os.path.join(trialsegs_output_folder, f"{subject_id}_{safe_label}-epo.fif"), overwrite=True)



In [ ]:
''' 
Subject 19!
'''

# Load your data
edf_file_path = 'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\raw-with-ICA-weights\\raw-with-ICA-weights\\sub-19_task-classicalMusic_eeg_raw-with-ICA-weights.fif' 
raw = mne.io.read_raw_fif(edf_file_path, preload=True)

# Define events manually based on step 2
# Here's a hypothetical example:
events_sub19 = [
    [start_trial_1_sub19, 0, 1],
    [end_trial_1_sub19, 0, 2],
    [start_trial_2_sub19, 0, 1],
    [end_trial_2_sub19, 0, 2],
    [start_trial_3_sub19, 0, 1],
    [end_trial_3_sub19, 0, 2],
    [start_trial_4_sub19, 0, 1],
    [end_trial_4_sub19, 0, 2],
    [start_trial_5_sub19, 0, 1],
    [end_trial_5_sub19, 0, 2],
    [start_trial_6_sub19, 0, 1],
    [end_trial_6_sub19, 0, 2],
    [start_trial_7_sub19, 0, 1],
    [end_trial_7_sub19, 0, 2],
    [start_trial_8_sub19, 0, 1],
    [end_trial_8_sub19, 0, 2],
    [start_trial_9_sub19, 0, 1],
    [end_trial_9_sub19, 0, 2],
    [start_trial_10_sub19, 0, 1],
    [end_trial_10_sub19, 0, 2],
    [start_trial_11_sub19, 0, 1],
    [end_trial_11_sub19, 0, 2],
    [start_trial_12_sub19, 0, 1],
    [end_trial_12_sub19, 0, 2]
]

# Create trial segmentations (trialsegs) based on these events
trial_starts_sub19 = [event[0] for event in events_sub19 if event[2] == 1]
trial_ends_sub19 = [event[0] for event in events_sub19 if event[2] == 2]

tmax_values_sub19 = [(end - start) / raw.info['sfreq'] for start, end in zip(trial_starts_sub19, trial_ends_sub19)]

trialsegs_sub19 = []
for i, start in enumerate(trial_starts_sub19):
    epochs_temp_sub19 = mne.Epochs(raw, events=[events_sub19[i*2]], event_id={'start': 1}, tmin=0, tmax=tmax_values_sub19[i], baseline=None, preload=True)
    trialsegs_sub19.append(epochs_temp_sub19)

# Manually label each trialseg
labels_sub19 = []
for ts in trialsegs_sub19:
    label = input(f"Enter label for trialseg {len(labels_sub19)+1}: ")  # This will prompt you to enter a label for each trialseg
    labels_sub19.append(label)

# Assign metadata to each individual mne.Epochs object
for i, ts in enumerate(trialsegs_sub19):
    ts.metadata = pd.DataFrame({'label': [labels_sub19[i]]})

# Create a folder for the subject's trialsegs if it doesn't exist
subject_id = "sub19"
trialsegs_output_folder = os.path.join("C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data", subject_id)
if not os.path.exists(trialsegs_output_folder):
    os.makedirs(trialsegs_output_folder)

# Save each labeled trialseg using its label as the name
for ts, label in zip(trialsegs_sub19, labels_sub19):
    safe_label = label.replace(" ", "_")  # to ensure filename compatibility
    ts.save(os.path.join(trialsegs_output_folder, f"{subject_id}_{safe_label}-epo.fif"), overwrite=True)


In [ ]:
''' 
Subject 21!
'''

# Load your data
edf_file_path = 'C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\raw-with-ICA-weights\\raw-with-ICA-weights\\sub-21_task-classicalMusic_eeg_raw-with-ICA-weights.fif' 
raw = mne.io.read_raw_fif(edf_file_path, preload=True)

# Define events manually based on step 2
# Here's a hypothetical example:
events_sub21 = [
    [start_trial_1_sub21, 0, 1],
    [end_trial_1_sub21, 0, 2],
    [start_trial_2_sub21, 0, 1],
    [end_trial_2_sub21, 0, 2],
    [start_trial_3_sub21, 0, 1],
    [end_trial_3_sub21, 0, 2],
    [start_trial_4_sub21, 0, 1],
    [end_trial_4_sub21, 0, 2],
    [start_trial_5_sub21, 0, 1],
    [end_trial_5_sub21, 0, 2],
    [start_trial_6_sub21, 0, 1],
    [end_trial_6_sub21, 0, 2],
    [start_trial_7_sub21, 0, 1],
    [end_trial_7_sub21, 0, 2],
    [start_trial_8_sub21, 0, 1],
    [end_trial_8_sub21, 0, 2],
    [start_trial_9_sub21, 0, 1],
    [end_trial_9_sub21, 0, 2],
    [start_trial_10_sub21, 0, 1],
    [end_trial_10_sub21, 0, 2],
    [start_trial_11_sub21, 0, 1],
    [end_trial_11_sub21, 0, 2],
    [start_trial_12_sub21, 0, 1],
    [end_trial_12_sub21, 0, 2]
]

# Create trial segmentations (trialsegs) based on these events
trial_starts_sub21 = [event[0] for event in events_sub21 if event[2] == 1]
trial_ends_sub21 = [event[0] for event in events_sub21 if event[2] == 2]

tmax_values_sub21 = [(end - start) / raw.info['sfreq'] for start, end in zip(trial_starts_sub21, trial_ends_sub21)]

trialsegs_sub21 = []
for i, start in enumerate(trial_starts_sub21):
    epochs_temp_sub21 = mne.Epochs(raw, events=[events_sub21[i*2]], event_id={'start': 1}, tmin=0, tmax=tmax_values_sub21[i], baseline=None, preload=True)
    trialsegs_sub21.append(epochs_temp_sub21)

# Manually label each trialseg
labels_sub21 = []
for ts in trialsegs_sub21:
    label = input(f"Enter label for trialseg {len(labels_sub21)+1}: ")  # This will prompt you to enter a label for each trialseg
    labels_sub21.append(label)

# Assign metadata to each individual mne.Epochs object
for i, ts in enumerate(trialsegs_sub21):
    ts.metadata = pd.DataFrame({'label': [labels_sub21[i]]})

# Create a folder for the subject's trialsegs if it doesn't exist
subject_id = "sub21"
trialsegs_output_folder = os.path.join("C:\\Users\\josep\\Music&Topology\\all work ;)\\Analysis1\\trialsegs_data", subject_id)
if not os.path.exists(trialsegs_output_folder):
    os.makedirs(trialsegs_output_folder)

# Save each labeled trialseg using its label as the name
for ts, label in zip(trialsegs_sub21, labels_sub21):
    safe_label = label.replace(" ", "_")  # to ensure filename compatibility
    ts.save(os.path.join(trialsegs_output_folder, f"{subject_id}_{safe_label}-epo.fif"), overwrite=True)

In [ ]:
# Define epochs for each subject based on the above visualization


- Continuing manually, patient-by-patient

In [ ]:
# Extract unique parent/subject directories from all_edf_files
subject_dirs_with_edf = list({file.parent for file in all_edf_files})

# Filter out directories that don't match the 'sub-XX' format
subject_dirs_with_edf = [dir for dir in subject_dirs_with_edf if 'sub-' in dir.name]

- below: older analyses, but leaving code for possible re-purposing later

In [ ]:
raw = mne.io.read_raw_edf('C:\\Users\\josep\\Music&Topology\\all-subjects\\sub-01\\eeg\\sub-01_task-classicalMusic_eeg.edf', preload=True)

# Define frequency bands
theta_band = (4, 7)
alpha_band = (8, 13)

# Bandpass filter the data for theta frequency band
raw_theta = raw.copy().filter(l_freq=theta_band[0], h_freq=theta_band[1])

# Bandpass filter the data for alpha frequency band
raw_alpha = raw.copy().filter(l_freq=alpha_band[0], h_freq=alpha_band[1])

# Visualize a representative channel for both filtered data
start, stop = raw.time_as_index([0, 1700])  # 6 seconds of data from time 194 to 200
times = raw.times[start:stop]

# Extracting data for channel 'Fp1'
theta_data, _ = raw_theta['Fp1', start:stop]
alpha_data, _ = raw_alpha['Fp1', start:stop]

# Plotting
plt.figure(figsize=(15, 6))

# Theta band
plt.subplot(2, 1, 1)
plt.plot(times, theta_data.T)
plt.title('Theta Filtered EEG Data for Channel Fp1 (194s to 200s)')
plt.ylabel('EEG amplitude (uV)')

# Alpha band
plt.subplot(2, 1, 2)
plt.plot(times, alpha_data.T)
plt.title('Alpha Filtered EEG Data for Channel Fp1 (194s to 200s)')
plt.xlabel('Time (s)')
plt.ylabel('EEG amplitude (uV)')

plt.tight_layout()
plt.show()